In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 画像を読み込む
IMAGE_PATH = "image/"
img = cv2.imread(IMAGE_PATH + "segment_test.png")

# 画像をグレースケールに変換
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# グレースケール画像を表示
plt.imshow(gray, cmap='gray')
plt.show()



入力される画像のセグメントがOnかOffかをパターンマッチングによって判定したいです。

画像のセグメントがOnかOffかをパターンマッチングで判定するためには、以下のような手順を踏むことが一般的です。ここではPythonとOpenCVを使用した例を示します。
画像の読み込み
グレースケール変換
二値化
パターンマッチング
以下にサンプルコードを示します。
このコードでは、入力画像と「On」および「Off」のテンプレート画像を比較し、最も一致するテンプレートに基づいてセグメントがOnかOffかを判定します。テンプレート画像は事前に用意しておく必要があります。
opencvインストールの仕方

OpenCVをインストールするには、以下の手順に従ってください。Pythonのパッケージマネージャであるpipを使用するのが一般的です。
pipを使用してインストールする方法:
Bash
Jupyter Notebookでインストールする方法:
Jupyter Notebookのセルに以下のコマンドを入力して実行します。
これでOpenCVがインストールされます。インストールが完了したら、以下のようにしてインポートできるか確認してください。
これでOpenCVが正しくインストールされているか確認できます。
7セグメントディスプレイの各セグメントが光っているか、光っていないかを判定したいです。

各セグメントの座標を記憶している正解パターンがあり、入力されるセグメントの画像のずれは十分小さいものとして、ほぼ同じ座標に各セグメントがあるものとします。
全点灯時のセグメントはsegment_A,segment_B..と各セグメントの座標と入力された画像の該当する座標のピクセルを比較し、ON、OFFを判定したいです

In [1]:
IMAGE_PATH = "image/"

In [6]:
import cv2
import numpy as np

image = cv2.imread('C:/Users/Owner/Desktop/my_program/my_pattern_match/image/segment_test.png')

In [8]:
IMAGE_PATH = 'C:/Users/Owner/Desktop/my_program/my_pattern_match/image/'

In [12]:
import cv2
import numpy as np

# 画像の読み込み
image = cv2.imread(IMAGE_PATH+'segment_test.png')

# グレースケール変換
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 二値化
_, binary = cv2.threshold(gray, 128, 0, cv2.THRESH_BINARY)

# 各セグメントの座標 (例: 各セグメントの中心座標)
segments = {
    'A': (100, 50),
    'B': (150, 100),
    'C': (150, 200),
    'D': (100, 250),
    'E': (50, 200),
    'F': (50, 100),
    'G': (100, 150)
}

# 各セグメントの状態を判定
segment_status = {}
for segment, (x, y) in segments.items():
    # 該当する座標のピクセル値を取得
    pixel_value = binary[y, x]
    # ピクセル値が0ならON、0ならOFFと判定
    if pixel_value == 0:
        segment_status[segment] = 'ON'
    else:
        segment_status[segment] = 'OFF'

# 結果を表示
for segment, status in segment_status.items():
    print(f"Segment {segment} is {status}")

Segment A is OFF
Segment B is ON
Segment C is OFF
Segment D is OFF
Segment E is OFF
Segment F is OFF
Segment G is OFF


## マウスクリックで座標を取得する

In [16]:
import cv2

# 画像の読み込み
image_path = 'segment_test.png'
image = cv2.imread(image_path)

# 画像が正しく読み込まれたか確認
if image is None:
    print(f"Error: 画像が見つかりません。パスを確認してください: {image_path}")
else:
    # 画像を表示し、クリックした座標を取得する関数
    def get_coordinates(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            print(f"クリックされた座標: ({x}, {y})")

    # ウィンドウを作成し、マウスコールバックを設定
    cv2.namedWindow('image')
    cv2.setMouseCallback('image', get_coordinates)

    try:
        # 画像を表示
        while True:
            cv2.imshow('image', image)
            if cv2.waitKey(20) & 0xFF == 27:  # ESCキーで終了
                break
    except KeyboardInterrupt:
        print("停止")
    cv2.destroyAllWindows()

クリックされた座標: (151, 166)
クリックされた座標: (149, 156)


## rev3 マウスドラッグで範囲指定

In [21]:
import cv2

# 画像の読み込み
image_path = 'segment_test.png'
image = cv2.imread(image_path)

# 画像が正しく読み込まれたか確認
if image is None:
    print(f"Error: 画像が見つかりません。パスを確認してください: {image_path}")
else:
    # グローバル変数
    ref_point = []
    cropping = False

    # マウスコールバック関数
    def shape_selection(event, x, y, flags, param):
        global ref_point, cropping

        if event == cv2.EVENT_LBUTTONDOWN:
            ref_point = [(x, y)]
            cropping = True

        elif event == cv2.EVENT_MOUSEMOVE:
            if cropping:
                temp_image = image.copy()
                cv2.rectangle(temp_image, ref_point[0], (x, y), (0, 0, 0), 2)
                cv2.imshow("image", temp_image)

        elif event == cv2.EVENT_LBUTTONUP:
            ref_point.append((x, y))
            cropping = False

            cv2.rectangle(image, ref_point[0], ref_point[1], (0,0, 0), 2)
            cv2.imshow("image", image)

    # ウィンドウを作成し、マウスコールバックを設定
    cv2.namedWindow("image")
    cv2.setMouseCallback("image", shape_selection)

    try:
        # 画像を表示
        while True:
            cv2.imshow("image", image)
            key = cv2.waitKey(1) & 0xFF

            if key == 27:  # ESCキーで終了
                break

        if len(ref_point) == 2:
            print(f"選択された範囲: 左上: {ref_point[0]}, 右下: {ref_point[1]}")

    except KeyboardInterrupt:
        print("プログラムが停止されました。")

    cv2.destroyAllWindows()

選択された範囲: 左上: (112, 113), 右下: (157, 125)


## rev4 矩形の形状を変更する

In [24]:
import cv2

# 画像の読み込み
image_path = 'segment_test.png'
image = cv2.imread(image_path)

# 画像が正しく読み込まれたか確認
if image is None:
    print(f"Error: 画像が見つかりません。パスを確認してください: {image_path}")
else:
    # グローバル変数
    ref_point = []
    cropping = False
    moving = False
    resizing = False
    selected_rect = None

    # マウスコールバック関数
    def shape_selection(event, x, y, flags, param):
        global ref_point, cropping, moving, resizing, selected_rect

        if event == cv2.EVENT_LBUTTONDOWN:
            if selected_rect and selected_rect[0] <= x <= selected_rect[2] and selected_rect[1] <= y <= selected_rect[3]:
                moving = True
                ref_point = [(x, y)]
            else:
                ref_point = [(x, y)]
                cropping = True
                selected_rect = None

        elif event == cv2.EVENT_MOUSEMOVE:
            if cropping:
                temp_image = image.copy()
                cv2.rectangle(temp_image, ref_point[0], (x, y), (0, 255, 0), 2)
                cv2.imshow("image", temp_image)
            elif moving:
                dx = x - ref_point[0][0]
                dy = y - ref_point[0][1]
                selected_rect = (selected_rect[0] + dx, selected_rect[1] + dy, selected_rect[2] + dx, selected_rect[3] + dy)
                ref_point = [(x, y)]
                temp_image = image.copy()
                cv2.rectangle(temp_image, (selected_rect[0], selected_rect[1]), (selected_rect[2], selected_rect[3]), (0, 255, 0), 2)
                cv2.imshow("image", temp_image)

        elif event == cv2.EVENT_LBUTTONUP:
            if cropping:
                ref_point.append((x, y))
                cropping = False
                selected_rect = (ref_point[0][0], ref_point[0][1], x, y)
                cv2.rectangle(image, ref_point[0], ref_point[1], (0, 255, 0), 2)
                cv2.imshow("image", image)
            elif moving:
                moving = False

    # ウィンドウを作成し、マウスコールバックを設定
    cv2.namedWindow("image")
    cv2.setMouseCallback("image", shape_selection)

    try:
        # 画像を表示
        while True:
            cv2.imshow("image", image)
            key = cv2.waitKey(1) & 0xFF

            if key == 27:  # ESCキーで終了
                break

        if selected_rect:
            print(f"選択された範囲: 左上: ({selected_rect[0]}, {selected_rect[1]}), 右下: ({selected_rect[2]}, {selected_rect[3]})")

    except KeyboardInterrupt:
        print("プログラムが停止されました。")

    cv2.destroyAllWindows()

選択された範囲: 左上: (327, 203), 右下: (400, 306)


In [25]:
import cv2

# 画像の読み込み
image_path = 'segment_test.png'
image = cv2.imread(image_path)

# 画像が正しく読み込まれたか確認
if image is None:
    print(f"Error: 画像が見つかりません。パスを確認してください: {image_path}")
else:
    # グローバル変数
    ref_point = []
    cropping = False
    moving = False
    selected_rect = None

    # マウスコールバック関数
    def shape_selection(event, x, y, flags, param):
        global ref_point, cropping, moving, selected_rect

        if event == cv2.EVENT_LBUTTONDOWN:
            if selected_rect and selected_rect[0] <= x <= selected_rect[2] and selected_rect[1] <= y <= selected_rect[3]:
                moving = True
                ref_point = [(x, y)]
            else:
                ref_point = [(x, y)]
                cropping = True
                selected_rect = None

        elif event == cv2.EVENT_MOUSEMOVE:
            if cropping:
                temp_image = image.copy()
                cv2.rectangle(temp_image, ref_point[0], (x, y), (0, 255, 0), 2)
                cv2.imshow("image", temp_image)
            elif moving:
                dx = x - ref_point[0][0]
                dy = y - ref_point[0][1]
                selected_rect = (selected_rect[0] + dx, selected_rect[1] + dy, selected_rect[2] + dx, selected_rect[3] + dy)
                ref_point = [(x, y)]
                temp_image = image.copy()
                cv2.rectangle(temp_image, (selected_rect[0], selected_rect[1]), (selected_rect[2], selected_rect[3]), (0, 255, 0), 2)
                cv2.imshow("image", temp_image)

        elif event == cv2.EVENT_LBUTTONUP:
            if cropping:
                ref_point.append((x, y))
                cropping = False
                selected_rect = (ref_point[0][0], ref_point[0][1], x, y)
                cv2.rectangle(image, ref_point[0], ref_point[1], (0, 255, 0), 2)
                cv2.imshow("image", image)
            elif moving:
                moving = False

    # ウィンドウを作成し、マウスコールバックを設定
    cv2.namedWindow("image")
    cv2.setMouseCallback("image", shape_selection)

    try:
        # 画像を表示
        while True:
            cv2.imshow("image", image)
            key = cv2.waitKey(1) & 0xFF

            if key == 27:  # ESCキーで終了
                break

        if selected_rect:
            print(f"選択された範囲: 左上: ({selected_rect[0]}, {selected_rect[1]}), 右下: ({selected_rect[2]}, {selected_rect[3]})")

    except KeyboardInterrupt:
        print("プログラムが停止されました。")

    cv2.destroyAllWindows()

選択された範囲: 左上: (236, 357), 右下: (113, 397)


## rev5 矩形領域　選択＆変形モード追加

In [26]:
import cv2

# 画像の読み込み
image_path = 'segment_test.png'
image = cv2.imread(image_path)

# 画像が正しく読み込まれたか確認
if image is None:
    print(f"Error: 画像が見つかりません。パスを確認してください: {image_path}")
else:
    # グローバル変数
    ref_point = []
    cropping = False
    moving = False
    selected_rect = None
    mode = 'select'  # 初期モードは選択モード

    # マウスコールバック関数
    def shape_selection(event, x, y, flags, param):
        global ref_point, cropping, moving, selected_rect, mode

        if mode == 'select':
            if event == cv2.EVENT_LBUTTONDOWN:
                ref_point = [(x, y)]
                cropping = True
                selected_rect = None

            elif event == cv2.EVENT_MOUSEMOVE:
                if cropping:
                    temp_image = image.copy()
                    cv2.rectangle(temp_image, ref_point[0], (x, y), (0, 255, 0), 2)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if cropping:
                    ref_point.append((x, y))
                    cropping = False
                    selected_rect = (ref_point[0][0], ref_point[0][1], x, y)
                    cv2.rectangle(image, ref_point[0], ref_point[1], (0, 255, 0), 2)
                    cv2.imshow("image", image)

        elif mode == 'move':
            if event == cv2.EVENT_LBUTTONDOWN:
                if selected_rect and selected_rect[0] <= x <= selected_rect[2] and selected_rect[1] <= y <= selected_rect[3]:
                    moving = True
                    ref_point = [(x, y)]

            elif event == cv2.EVENT_MOUSEMOVE:
                if moving:
                    dx = x - ref_point[0][0]
                    dy = y - ref_point[0][1]
                    selected_rect = (selected_rect[0] + dx, selected_rect[1] + dy, selected_rect[2] + dx, selected_rect[3] + dy)
                    ref_point = [(x, y)]
                    temp_image = image.copy()
                    cv2.rectangle(temp_image, (selected_rect[0], selected_rect[1]), (selected_rect[2], selected_rect[3]), (0, 255, 0), 2)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if moving:
                    moving = False

    # ウィンドウを作成し、マウスコールバックを設定
    cv2.namedWindow("image")
    cv2.setMouseCallback("image", shape_selection)

    try:
        # 画像を表示
        while True:
            cv2.imshow("image", image)
            key = cv2.waitKey(1) & 0xFF

            if key == 27:  # ESCキーで終了
                break
            elif key == ord('s'):  # 's'キーで選択モードに切り替え
                mode = 'select'
                print("選択モード")
            elif key == ord('m'):  # 'm'キーで変形モードに切り替え
                mode = 'move'
                print("変形モード")

        if selected_rect:
            print(f"選択された範囲: 左上: ({selected_rect[0]}, {selected_rect[1]}), 右下: ({selected_rect[2]}, {selected_rect[3]})")

    except KeyboardInterrupt:
        print("プログラムが停止されました。")

    cv2.destroyAllWindows()

選択モード
変形モード
選択された範囲: 左上: (164, 73), 右下: (187, 127)


## rev6 変形モードの画像位置を修正

In [32]:
import cv2

# 画像の読み込み
image_path = 'segment_test.png'
image = cv2.imread(image_path)

# 画像が正しく読み込まれたか確認
if image is None:
    print(f"Error: 画像が見つかりません。パスを確認してください: {image_path}")
else:
    # グローバル変数
    ref_point = []
    cropping = False
    moving = False
    selected_rect = None
    mode = 'select'  # 初期モードは選択モード

    # マウスコールバック関数
    def shape_selection(event, x, y, flags, param):
        global ref_point, cropping, moving, selected_rect, mode

        if mode == 'select':
            if event == cv2.EVENT_LBUTTONDOWN:
                ref_point = [(x, y)]
                cropping = True
                selected_rect = None

            elif event == cv2.EVENT_MOUSEMOVE:
                if cropping:
                    temp_image = image.copy()
                    cv2.rectangle(temp_image, ref_point[0], (x, y), (0, 255, 0), 2)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if cropping:
                    ref_point.append((x, y))
                    cropping = False
                    selected_rect = (ref_point[0][0], ref_point[0][1], x, y)
                    cv2.rectangle(image, ref_point[0], ref_point[1], (0, 255, 0), 2)
                    cv2.imshow("image", image)

        elif mode == 'move':
            if event == cv2.EVENT_LBUTTONDOWN:
                if selected_rect and selected_rect[0] <= x <= selected_rect[2] and selected_rect[1] <= y <= selected_rect[3]:
                    moving = True
                    ref_point = [(x, y)]

            elif event == cv2.EVENT_MOUSEMOVE:
                if moving:
                    dx = x - ref_point[0][0]
                    dy = y - ref_point[0][1]
                    temp_image = image.copy()
                    new_rect = (selected_rect[0] + dx, selected_rect[1] + dy, selected_rect[2] + dx, selected_rect[3] + dy)
                    cv2.rectangle(temp_image, (new_rect[0], new_rect[1]), (new_rect[2], new_rect[3]), (0, 255, 0), 2)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if moving:
                    dx = x - ref_point[0][0]
                    dy = y - ref_point[0][1]
                    selected_rect = (selected_rect[0] + dx, selected_rect[1] + dy, selected_rect[2] + dx, selected_rect[3] + dy)
                    moving = False
                    cv2.rectangle(image, (selected_rect[0], selected_rect[1]), (selected_rect[2], selected_rect[3]), (0, 255, 0), 2)
                    cv2.imshow("image", image)

    # ウィンドウを作成し、マウスコールバックを設定
    cv2.namedWindow("image")
    cv2.setMouseCallback("image", shape_selection)

    try:
        # 画像を表示
        while True:
            cv2.imshow("image", image)
            key = cv2.waitKey(1) & 0xFF

            if key == 27:  # ESCキーで終了
                break
            elif key == ord('s'):  # 's'キーで選択モードに切り替え
                mode = 'select'
                print("選択モード")
            elif key == ord('m'):  # 'm'キーで変形モードに切り替え
                mode = 'move'
                print("変形モード")

        if selected_rect:
            print(f"選択された範囲: 左上: ({selected_rect[0]}, {selected_rect[1]}), 右下: ({selected_rect[2]}, {selected_rect[3]})")

    except KeyboardInterrupt:
        print("プログラムが停止されました。")

    cv2.destroyAllWindows()

変形モード
選択された範囲: 左上: (46, 58), 右下: (247, 220)


## rev7 モード変更で過去の矩形を削除

In [35]:
import cv2

# 画像の読み込み
image_path = 'segment_test.png'
image = cv2.imread(image_path)

# 画像が正しく読み込まれたか確認
if image is None:
    print(f"Error: 画像が見つかりません。パスを確認してください: {image_path}")
else:
    # グローバル変数
    ref_point = []
    cropping = False
    moving = False
    selected_rect = None
    mode = 'select'  # 初期モードは選択モード
    original_image = image.copy()  # 元の画像を保持

    # マウスコールバック関数
    def shape_selection(event, x, y, flags, param):
        global ref_point, cropping, moving, selected_rect, mode, image

        if mode == 'select':
            if event == cv2.EVENT_LBUTTONDOWN:
                ref_point = [(x, y)]
                cropping = True
                selected_rect = None

            elif event == cv2.EVENT_MOUSEMOVE:
                if cropping:
                    temp_image = image.copy()
                    cv2.rectangle(temp_image, ref_point[0], (x, y), (0, 255, 0), 2)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if cropping:
                    ref_point.append((x, y))
                    cropping = False
                    selected_rect = (ref_point[0][0], ref_point[0][1], x, y)
                    image = original_image.copy()  # 元の画像に戻す
                    cv2.rectangle(image, ref_point[0], ref_point[1], (0, 255, 0), 2)
                    cv2.imshow("image", image)

        elif mode == 'move':
            if event == cv2.EVENT_LBUTTONDOWN:
                if selected_rect and selected_rect[0] <= x <= selected_rect[2] and selected_rect[1] <= y <= selected_rect[3]:
                    moving = True
                    ref_point = [(x, y)]

            elif event == cv2.EVENT_MOUSEMOVE:
                if moving:
                    dx = x - ref_point[0][0]
                    dy = y - ref_point[0][1]
                    temp_image = original_image.copy()  # 元の画像に戻す
                    new_rect = (selected_rect[0] + dx, selected_rect[1] + dy, selected_rect[2] + dx, selected_rect[3] + dy)
                    cv2.rectangle(temp_image, (new_rect[0], new_rect[1]), (new_rect[2], new_rect[3]), (0, 255, 0), 2)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if moving:
                    dx = x - ref_point[0][0]
                    dy = y - ref_point[0][1]
                    selected_rect = (selected_rect[0] + dx, selected_rect[1] + dy, selected_rect[2] + dx, selected_rect[3] + dy)
                    moving = False
                    image = original_image.copy()  # 元の画像に戻す
                    cv2.rectangle(image, (selected_rect[0], selected_rect[1]), (selected_rect[2], selected_rect[3]), (0, 255, 0), 2)
                    cv2.imshow("image", image)

    # ウィンドウを作成し、マウスコールバックを設定
    cv2.namedWindow("image")
    cv2.setMouseCallback("image", shape_selection)

    try:
        # 画像を表示
        while True:
            cv2.imshow("image", image)
            key = cv2.waitKey(1) & 0xFF

            if key == 27:  # ESCキーで終了
                break
            elif key == ord('s'):  # 's'キーで選択モードに切り替え
                mode = 'select'
                image = original_image.copy()  # 元の画像に戻す
                print("選択モード")
            elif key == ord('m'):  # 'm'キーで移動モードに切り替え
                mode = 'move'
                image = original_image.copy()  # 元の画像に戻す
                print("移動モード")

        if selected_rect:
            print(f"選択された範囲: 左上: ({selected_rect[0]}, {selected_rect[1]}), 右下: ({selected_rect[2]}, {selected_rect[3]})")

    except KeyboardInterrupt:
        print("プログラムが停止されました。")

    cv2.destroyAllWindows()

変形モード
選択された範囲: 左上: (34, 49), 右下: (297, 245)


## rev9 モード以降時の矩形のバグを修正

In [36]:
import cv2

# 画像の読み込み
image_path = 'segment_test.png'
image = cv2.imread(image_path)

# 画像が正しく読み込まれたか確認
if image is None:
    print(f"Error: 画像が見つかりません。パスを確認してください: {image_path}")
else:
    # グローバル変数
    ref_point = []
    cropping = False
    moving = False
    selected_rect = None
    mode = 'select'  # 初期モードは選択モード
    original_image = image.copy()  # 元の画像を保持

    # マウスコールバック関数
    def shape_selection(event, x, y, flags, param):
        global ref_point, cropping, moving, selected_rect, mode, image

        if mode == 'select':
            if event == cv2.EVENT_LBUTTONDOWN:
                ref_point = [(x, y)]
                cropping = True
                selected_rect = None

            elif event == cv2.EVENT_MOUSEMOVE:
                if cropping:
                    temp_image = image.copy()
                    cv2.rectangle(temp_image, ref_point[0], (x, y), (0, 255, 0), 2)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if cropping:
                    ref_point.append((x, y))
                    cropping = False
                    selected_rect = (ref_point[0][0], ref_point[0][1], x, y)
                    image = original_image.copy()  # 元の画像に戻す
                    cv2.rectangle(image, ref_point[0], ref_point[1], (0, 255, 0), 2)
                    cv2.imshow("image", image)

        elif mode == 'move':
            if event == cv2.EVENT_LBUTTONDOWN:
                if selected_rect and selected_rect[0] <= x <= selected_rect[2] and selected_rect[1] <= y <= selected_rect[3]:
                    moving = True
                    ref_point = [(x, y)]
                    image = original_image.copy()  # 元の画像に戻す

            elif event == cv2.EVENT_MOUSEMOVE:
                if moving:
                    dx = x - ref_point[0][0]
                    dy = y - ref_point[0][1]
                    temp_image = original_image.copy()  # 元の画像に戻す
                    new_rect = (selected_rect[0] + dx, selected_rect[1] + dy, selected_rect[2] + dx, selected_rect[3] + dy)
                    cv2.rectangle(temp_image, (new_rect[0], new_rect[1]), (new_rect[2], new_rect[3]), (0, 255, 0), 2)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if moving:
                    dx = x - ref_point[0][0]
                    dy = y - ref_point[0][1]
                    selected_rect = (selected_rect[0] + dx, selected_rect[1] + dy, selected_rect[2] + dx, selected_rect[3] + dy)
                    moving = False
                    image = original_image.copy()  # 元の画像に戻す
                    cv2.rectangle(image, (selected_rect[0], selected_rect[1]), (selected_rect[2], selected_rect[3]), (0, 255, 0), 2)
                    cv2.imshow("image", image)

    # ウィンドウを作成し、マウスコールバックを設定
    cv2.namedWindow("image")
    cv2.setMouseCallback("image", shape_selection)

    try:
        # 画像を表示
        while True:
            cv2.imshow("image", image)
            key = cv2.waitKey(1) & 0xFF

            if key == 27:  # ESCキーで終了
                break
            elif key == ord('s'):  # 's'キーで選択モードに切り替え
                mode = 'select'
                print("選択モード")
            elif key == ord('m'):  # 'm'キーで変形モードに切り替え
                mode = 'move'
                print("変形モード")

        if selected_rect:
            print(f"選択された範囲: 左上: ({selected_rect[0]}, {selected_rect[1]}), 右下: ({selected_rect[2]}, {selected_rect[3]})")

    except KeyboardInterrupt:
        print("プログラムが停止されました。")

    cv2.destroyAllWindows()

変形モード
選択された範囲: 左上: (48, 65), 右下: (324, 214)


## rev10 矩形表示を7セグメント化

In [38]:
import cv2
import numpy as np

# 画像の読み込み
image_path = 'segment_test.png'
image = cv2.imread(image_path)

# 画像が正しく読み込まれたか確認
if image is None:
    print(f"Error: 画像が見つかりません。パスを確認してください: {image_path}")
else:
    # 7セグメントディスプレイの各セグメントの座標を定義
    segments = {
        'A': [(50, 20), (150, 40)],
        'B': [(160, 50), (180, 150)],
        'C': [(160, 160), (180, 260)],
        'D': [(50, 270), (150, 290)],
        'E': [(30, 160), (50, 260)],
        'F': [(30, 50), (50, 150)],
        'G': [(50, 140), (150, 160)]
    }

    # 各セグメントを描画
    for segment, ((x1, y1), (x2, y2)) in segments.items():
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # 画像を表示
    cv2.imshow("7-Segment Display", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

## rev11 7セグメントをよりリアルに描画

In [39]:
import cv2
import numpy as np

# 画像の読み込み
image_path = 'segment_test.png'
image = cv2.imread(image_path)

# 画像が正しく読み込まれたか確認
if image is None:
    print(f"Error: 画像が見つかりません。パスを確認してください: {image_path}")
else:
    # 7セグメントディスプレイの各セグメントの座標を定義
    segments = {
        'A': np.array([[50, 20], [150, 20], [140, 40], [60, 40]], np.int32),
        'B': np.array([[150, 20], [170, 40], [170, 140], [150, 130]], np.int32),
        'C': np.array([[150, 150], [170, 160], [170, 260], [150, 260]], np.int32),
        'D': np.array([[50, 270], [150, 270], [140, 290], [60, 290]], np.int32),
        'E': np.array([[30, 150], [50, 160], [50, 260], [30, 260]], np.int32),
        'F': np.array([[30, 20], [50, 20], [50, 130], [30, 140]], np.int32),
        'G': np.array([[50, 140], [150, 140], [140, 160], [60, 160]], np.int32),
        'DP': np.array([[180, 280], [200, 280], [200, 300], [180, 300]], np.int32)  # 小数点のドット
    }

    # 各セグメントを描画
    for segment, points in segments.items():
        cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)

    # 画像を表示
    cv2.imshow("7-Segment Display", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

## rev12 セグメント形状変更

In [40]:
import cv2
import numpy as np

# 画像の読み込み
image_path = 'segment_test.png'
image = cv2.imread(image_path)

# 画像が正しく読み込まれたか確認
if image is None:
    print(f"Error: 画像が見つかりません。パスを確認してください: {image_path}")
else:
    # 7セグメントディスプレイの各セグメントの座標を定義
    segments = {
        'A': np.array([[50, 20], [140, 20], [160, 40], [140, 60], [50, 60], [30, 40]], np.int32),
        'B': np.array([[140, 60], [160, 80], [160, 180], [140, 200], [120, 180], [120, 80]], np.int32),
        'C': np.array([[140, 200], [160, 220], [160, 320], [140, 340], [120, 320], [120, 220]], np.int32),
        'D': np.array([[50, 340], [140, 340], [160, 360], [140, 380], [50, 380], [30, 360]], np.int32),
        'E': np.array([[30, 200], [50, 220], [50, 320], [30, 340], [10, 320], [10, 220]], np.int32),
        'F': np.array([[30, 60], [50, 80], [50, 180], [30, 200], [10, 180], [10, 80]], np.int32),
        'G': np.array([[50, 180], [140, 180], [160, 200], [140, 220], [50, 220], [30, 200]], np.int32),
        'DP': np.array([[180, 360], [200, 360], [200, 380], [180, 380]], np.int32)  # 小数点のドット
    }

    # 各セグメントを描画
    for segment, points in segments.items():
        cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)

    # 画像を表示
    cv2.imshow("7-Segment Display", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

## rev13 セグメント微修正

In [75]:
import cv2
import numpy as np

# 画像の読み込み
image_path = 'segment_test.png'
image = cv2.imread(image_path)

# 画像が正しく読み込まれたか確認
if image is None:
    print(f"Error: 画像が見つかりません。パスを確認してください: {image_path}")
else:
    # 7セグメントディスプレイの各セグメントの座標を定義
    segments = {
        'A': np.array([[50, 40], [140, 40], [160, 60], [140, 80], [50, 80], [30, 60]], np.int32),
        'B': np.array([[160, 60], [180, 80], [180, 180], [160, 200], [140,180], [140, 80]], np.int32),
        'C': np.array([[160, 200], [180, 220], [180, 320], [160, 340], [140, 320], [140, 220]], np.int32),
        'D': np.array([[50, 320], [140, 320], [160, 340], [140, 360], [50, 360], [30, 340]], np.int32),
        'E': np.array([[30, 200], [50, 220], [50, 320], [30, 340], [10, 320], [10, 220]], np.int32),
        'F': np.array([[30, 60], [50, 80], [50, 180], [30, 200], [10, 180], [10, 80]], np.int32),
        'G': np.array([[50, 180], [140, 180], [160, 200], [140, 220], [50, 220], [30, 200]], np.int32),
        'DP': np.array([[180, 360], [200, 360], [200, 380], [180, 380]], np.int32)  # 小数点のドット
    }

    # 各セグメントを描画
    for segment, points in segments.items():
        cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)

    # 画像を表示
    cv2.imshow("7-Segment Display", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

## 【TBD】rev14 セグメントを右に傾ける

### セグメントの形は後日修正 TBD!

In [74]:
import cv2
import numpy as np

# 画像の読み込み
image_path = 'segment_test.png'
image = cv2.imread(image_path)

# 画像が正しく読み込まれたか確認
if image is None:
    print(f"Error: 画像が見つかりません。パスを確認してください: {image_path}")
else:
    # 7セグメントディスプレイの各セグメントの座標を定義
    segments = {
        'A': np.array([[50, 40], [140, 40], [160, 60], [140, 80], [50, 80], [30, 60]], np.int32),
        'B': np.array([[160, 60], [180, 80], [180, 180], [160, 200], [140, 180], [140, 80]], np.int32),
        'C': np.array([[160, 200], [180, 220], [180, 320], [160, 340], [140, 320], [140, 220]], np.int32),
        'D': np.array([[50-20, 320], [140-20, 320], [160-20, 340], [140-20, 360], [50-20, 360], [30-20, 340]], np.int32),
        'E': np.array([[30, 200], [50, 220], [50, 320], [30, 340], [10, 320], [10, 220]], np.int32),
        'F': np.array([[30, 60], [50, 80], [50, 180], [30, 200], [10, 180], [10, 80]], np.int32),
        'G': np.array([[50, 180], [140, 180], [160, 200], [140, 220], [50, 220], [30, 200]], np.int32),
        'DP': np.array([[180, 360], [200, 360], [200, 380], [180, 380]], np.int32)  # 小数点のドット
    }

    # 傾けるための変換行列を定義
    def skew_segment(points, skew_factor):
        skewed_points = []
        for x, y in points:
            skewed_points.append([x + skew_factor * (y - 40), y])
        return np.array(skewed_points, np.int32)

    # 傾けるセグメントのリスト
    skew_segments = ['B', 'C', 'E', 'F']

    # 傾ける量
    skew_factor = -0.3

    # 各セグメントを描画
    for segment, points in segments.items():
        if segment in skew_segments:
            points = skew_segment(points, skew_factor)
        cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)

    # 画像を表示
    cv2.imshow("7-Segment Display", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

## rev 15 セグメントサイズ縮小

In [79]:
import cv2
import numpy as np

# 画像の読み込み
image_path = 'segment_test.png'
image = cv2.imread(image_path)

# 画像が正しく読み込まれたか確認
if image is None:
    print(f"Error: 画像が見つかりません。パスを確認してください: {image_path}")
else:
    # 7セグメントディスプレイの各セグメントの座標を定義
    segments = {
        'A': np.array([[50, 40], [140, 40], [160, 60], [140, 80], [50, 80], [30, 60]], np.int32),
        'B': np.array([[160, 60], [180, 80], [180, 180], [160, 200], [140, 180], [140, 80]], np.int32),
        'C': np.array([[160, 200], [180, 220], [180, 320], [160, 340], [140, 320], [140, 220]], np.int32),
        'D': np.array([[50, 320], [140, 320], [160, 340], [140, 360], [50, 360], [30, 340]], np.int32),
        'E': np.array([[30, 200], [50, 220], [50, 320], [30, 340], [10, 320], [10, 220]], np.int32),
        'F': np.array([[30, 60], [50, 80], [50, 180], [30, 200], [10, 180], [10, 80]], np.int32),
        'G': np.array([[50, 180], [140, 180], [160, 200], [140, 220], [50, 220], [30, 200]], np.int32),
        'DP': np.array([[180, 360], [200, 360], [200, 380], [180, 380]], np.int32)  # 小数点のドット
    }

    # スケーリングと中央に寄せるための変換行列を定義
    def scale_and_center_segment(points, scale_factor, offset_x, offset_y):
        scaled_points = []
        for x, y in points:
            scaled_x = int(x * scale_factor + offset_x)
            scaled_y = int(y * scale_factor + offset_y)
            scaled_points.append([scaled_x, scaled_y])
        return np.array(scaled_points, np.int32)

    # スケーリング係数とオフセット
    scale_factor = 0.5
    offset_x = 100
    offset_y = 100

    # 各セグメントを描画
    for segment, points in segments.items():
        points = scale_and_center_segment(points, scale_factor, offset_x, offset_y)
        cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)

    # 画像を表示
    cv2.imshow("7-Segment Display", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

## rev16矩形をセグメントとして表示

In [80]:
import cv2
import numpy as np

# 画像の読み込み
image_path = 'segment_test.png'
image = cv2.imread(image_path)

# 画像が正しく読み込まれたか確認
if image is None:
    print(f"Error: 画像が見つかりません。パスを確認してください: {image_path}")
else:
    # グローバル変数
    ref_point = []
    cropping = False
    moving = False
    selected_rect = None
    mode = 'select'  # 初期モードは選択モード
    original_image = image.copy()  # 元の画像を保持

    # 7セグメントディスプレイの各セグメントの座標を定義
    segments = {
        'A': np.array([[50, 40], [140, 40], [160, 60], [140, 80], [50, 80], [30, 60]], np.int32),
        'B': np.array([[160, 60], [180, 80], [180, 180], [160, 200], [140, 180], [140, 80]], np.int32),
        'C': np.array([[160, 200], [180, 220], [180, 320], [160, 340], [140, 320], [140, 220]], np.int32),
        'D': np.array([[50, 320], [140, 320], [160, 340], [140, 360], [50, 360], [30, 340]], np.int32),
        'E': np.array([[30, 200], [50, 220], [50, 320], [30, 340], [10, 320], [10, 220]], np.int32),
        'F': np.array([[30, 60], [50, 80], [50, 180], [30, 200], [10, 180], [10, 80]], np.int32),
        'G': np.array([[50, 180], [140, 180], [160, 200], [140, 220], [50, 220], [30, 200]], np.int32),
        'DP': np.array([[180, 360], [200, 360], [200, 380], [180, 380]], np.int32)  # 小数点のドット
    }

    # スケーリングと中央に寄せるための変換行列を定義
    def scale_and_center_segment(points, scale_factor, offset_x, offset_y):
        scaled_points = []
        for x, y in points:
            scaled_x = int(x * scale_factor + offset_x)
            scaled_y = int(y * scale_factor + offset_y)
            scaled_points.append([scaled_x, scaled_y])
        return np.array(scaled_points, np.int32)

    # スケーリング係数とオフセット
    scale_factor = 0.5
    offset_x = 100
    offset_y = 100

    # スケーリングと中央に寄せたセグメントを保持する辞書
    scaled_segments = {segment: scale_and_center_segment(points, scale_factor, offset_x, offset_y) for segment, points in segments.items()}

    # マウスコールバック関数
    def shape_selection(event, x, y, flags, param):
        global ref_point, cropping, moving, selected_rect, mode, image

        if mode == 'select':
            if event == cv2.EVENT_LBUTTONDOWN:
                ref_point = [(x, y)]
                cropping = True
                selected_rect = None

            elif event == cv2.EVENT_MOUSEMOVE:
                if cropping:
                    temp_image = image.copy()
                    for points in scaled_segments.values():
                        cv2.polylines(temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    cv2.rectangle(temp_image, ref_point[0], (x, y), (0, 255, 0), 2)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if cropping:
                    ref_point.append((x, y))
                    cropping = False
                    selected_rect = (ref_point[0][0], ref_point[0][1], x, y)
                    image = original_image.copy()  # 元の画像に戻す
                    for points in scaled_segments.values():
                        cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    cv2.rectangle(image, ref_point[0], ref_point[1], (0, 255, 0), 2)
                    cv2.imshow("image", image)

        elif mode == 'move':
            if event == cv2.EVENT_LBUTTONDOWN:
                if selected_rect and selected_rect[0] <= x <= selected_rect[2] and selected_rect[1] <= y <= selected_rect[3]:
                    moving = True
                    ref_point = [(x, y)]
                    image = original_image.copy()  # 元の画像に戻す

            elif event == cv2.EVENT_MOUSEMOVE:
                if moving:
                    dx = x - ref_point[0][0]
                    dy = y - ref_point[0][1]
                    temp_image = original_image.copy()  # 元の画像に戻す
                    new_rect = (selected_rect[0] + dx, selected_rect[1] + dy, selected_rect[2] + dx, selected_rect[3] + dy)
                    for points in scaled_segments.values():
                        cv2.polylines(temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    cv2.rectangle(temp_image, (new_rect[0], new_rect[1]), (new_rect[2], new_rect[3]), (0, 255, 0), 2)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if moving:
                    dx = x - ref_point[0][0]
                    dy = y - ref_point[0][1]
                    selected_rect = (selected_rect[0] + dx, selected_rect[1] + dy, selected_rect[2] + dx, selected_rect[3] + dy)
                    moving = False
                    image = original_image.copy()  # 元の画像に戻す
                    for points in scaled_segments.values():
                        cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    cv2.rectangle(image, (selected_rect[0], selected_rect[1]), (selected_rect[2], selected_rect[3]), (0, 255, 0), 2)
                    cv2.imshow("image", image)

    # ウィンドウを作成し、マウスコールバックを設定
    cv2.namedWindow("image")
    cv2.setMouseCallback("image", shape_selection)

    try:
        # 画像を表示
        while True:
            cv2.imshow("image", image)
            key = cv2.waitKey(1) & 0xFF

            if key == 27:  # ESCキーで終了
                break
            elif key == ord('s'):  # 's'キーで選択モードに切り替え
                mode = 'select'
                print("選択モード")
            elif key == ord('m'):  # 'm'キーで変形モードに切り替え
                mode = 'move'
                print("変形モード")

        if selected_rect:
            print(f"選択された範囲: 左上: ({selected_rect[0]}, {selected_rect[1]}), 右下: ({selected_rect[2]}, {selected_rect[3]})")

    except KeyboardInterrupt:
        print("プログラムが停止されました。")

    cv2.destroyAllWindows()

変形モード
選択モード
選択された範囲: 左上: (246, 211), 右下: (434, 284)


## rev17 表示される矩形を7セグに変更

In [83]:
import cv2
import numpy as np

# 画像の読み込み
image_path = 'segment_test.png'
image = cv2.imread(image_path)

# 画像が正しく読み込まれたか確認
if image is None:
    print(f"Error: 画像が見つかりません。パスを確認してください: {image_path}")
else:
    # グローバル変数
    ref_point = []
    cropping = False
    moving = False
    selected_segment = None
    mode = 'select'  # 初期モードは選択モード
    original_image = image.copy()  # 元の画像を保持

    # 7セグメントディスプレイの各セグメントの座標を定義
    segments = {
        'A': np.array([[50, 40], [140, 40], [160, 60], [140, 80], [50, 80], [30, 60]], np.int32),
        'B': np.array([[160, 60], [180, 80], [180, 180], [160, 200], [140, 180], [140, 80]], np.int32),
        'C': np.array([[160, 200], [180, 220], [180, 320], [160, 340], [140, 320], [140, 220]], np.int32),
        'D': np.array([[50, 320], [140, 320], [160, 340], [140, 360], [50, 360], [30, 340]], np.int32),
        'E': np.array([[30, 200], [50, 220], [50, 320], [30, 340], [10, 320], [10, 220]], np.int32),
        'F': np.array([[30, 60], [50, 80], [50, 180], [30, 200], [10, 180], [10, 80]], np.int32),
        'G': np.array([[50, 180], [140, 180], [160, 200], [140, 220], [50, 220], [30, 200]], np.int32),
        'DP': np.array([[180, 360], [200, 360], [200, 380], [180, 380]], np.int32)  # 小数点のドット
    }

    # スケーリングと中央に寄せるための変換行列を定義
    def scale_and_center_segment(points, scale_factor, offset_x, offset_y):
        scaled_points = []
        for x, y in points:
            scaled_x = int(x * scale_factor + offset_x)
            scaled_y = int(y * scale_factor + offset_y)
            scaled_points.append([scaled_x, scaled_y])
        return np.array(scaled_points, np.int32)

    # スケーリング係数とオフセット
    scale_factor = 0.5
    offset_x = 100
    offset_y = 100

    # スケーリングと中央に寄せたセグメントを保持する辞書
    scaled_segments = {segment: scale_and_center_segment(points, scale_factor, offset_x, offset_y) for segment, points in segments.items()}

    # マウスコールバック関数
    def shape_selection(event, x, y, flags, param):
        global ref_point, cropping, moving, selected_segment, mode, image

        if mode == 'select':
            if event == cv2.EVENT_LBUTTONDOWN:
                ref_point = [(x, y)]
                cropping = True
                selected_segment = None

            elif event == cv2.EVENT_MOUSEMOVE:
                if cropping:
                    temp_image = image.copy()
                    for points in scaled_segments.values():
                        cv2.polylines(temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if cropping:
                    ref_point.append((x, y))
                    cropping = False
                    for segment, points in scaled_segments.items():
                        if cv2.pointPolygonTest(points, ref_point[0], False) >= 0:
                            selected_segment = segment
                            break
                    image = original_image.copy()  # 元の画像に戻す
                    for points in scaled_segments.values():
                        cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    if selected_segment:
                        cv2.polylines(image, [scaled_segments[selected_segment]], isClosed=True, color=(0, 0, 255), thickness=2)
                    cv2.imshow("image", image)

        elif mode == 'move':
            if event == cv2.EVENT_LBUTTONDOWN:
                if selected_segment and cv2.pointPolygonTest(scaled_segments[selected_segment], (x, y), False) >= 0:
                    moving = True
                    ref_point = [(x, y)]
                    image = original_image.copy()  # 元の画像に戻す

            elif event == cv2.EVENT_MOUSEMOVE:
                if moving:
                    dx = x - ref_point[0][0]
                    dy = y - ref_point[0][1]
                    temp_image = original_image.copy()  # 元の画像に戻す
                    new_points = scaled_segments[selected_segment] + [dx, dy]
                    for segment, points in scaled_segments.items():
                        if segment == selected_segment:
                            cv2.polylines(temp_image, [new_points], isClosed=True, color=(0, 0, 255), thickness=2)
                        else:
                            cv2.polylines(temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if moving:
                    dx = x - ref_point[0][0]
                    dy = y - ref_point[0][1]
                    scaled_segments[selected_segment] = scaled_segments[selected_segment] + [dx, dy]
                    moving = False
                    image = original_image.copy()  # 元の画像に戻す
                    for segment, points in scaled_segments.items():
                        if segment == selected_segment:
                            cv2.polylines(image, [points], isClosed=True, color=(0, 0, 255), thickness=2)
                        else:
                            cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    cv2.imshow("image", image)

    # ウィンドウを作成し、マウスコールバックを設定
    cv2.namedWindow("image")
    cv2.setMouseCallback("image", shape_selection)

    try:
        # 画像を表示
        while True:
            cv2.imshow("image", image)
            key = cv2.waitKey(1) & 0xFF

            if key == 27:  # ESCキーで終了
                break
            elif key == ord('s'):  # 's'キーで選択モードに切り替え
                mode = 'select'
                print("選択モード")
            elif key == ord('m'):  # 'm'キーで変形モードに切り替え
                mode = 'move'
                print("変形モード")

        if selected_segment:
            print(f"選択されたセグメント: {selected_segment}")

    except KeyboardInterrupt:
        print("プログラムが停止されました。")

    cv2.destroyAllWindows()

選択モード
変形モード
変形モード
選択モード
選択されたセグメント: B


## rev18【TBD】 移動モードをすべてのセグメント単位とする
※移動の仕方がきもいが、いったん置く

In [86]:
import cv2
import numpy as np

# 画像の読み込み
image_path = 'segment_test.png'
image = cv2.imread(image_path)

# 画像が正しく読み込まれたか確認
if image is None:
    print(f"Error: 画像が見つかりません。パスを確認してください: {image_path}")
else:
    # グローバル変数
    ref_point = []
    cropping = False
    moving = False
    selected_segment = None
    mode = 'select'  # 初期モードは選択モード
    original_image = image.copy()  # 元の画像を保持

    # 7セグメントディスプレイの各セグメントの座標を定義
    segments = {
        'A': np.array([[50, 40], [140, 40], [160, 60], [140, 80], [50, 80], [30, 60]], np.int32),
        'B': np.array([[160, 60], [180, 80], [180, 180], [160, 200], [140, 180], [140, 80]], np.int32),
        'C': np.array([[160, 200], [180, 220], [180, 320], [160, 340], [140, 320], [140, 220]], np.int32),
        'D': np.array([[50, 320], [140, 320], [160, 340], [140, 360], [50, 360], [30, 340]], np.int32),
        'E': np.array([[30, 200], [50, 220], [50, 320], [30, 340], [10, 320], [10, 220]], np.int32),
        'F': np.array([[30, 60], [50, 80], [50, 180], [30, 200], [10, 180], [10, 80]], np.int32),
        'G': np.array([[50, 180], [140, 180], [160, 200], [140, 220], [50, 220], [30, 200]], np.int32),
        'DP': np.array([[180, 360], [200, 360], [200, 380], [180, 380]], np.int32)  # 小数点のドット
    }

    # スケーリングと中央に寄せるための変換行列を定義
    def scale_and_center_segment(points, scale_factor, offset_x, offset_y):
        scaled_points = []
        for x, y in points:
            scaled_x = int(x * scale_factor + offset_x)
            scaled_y = int(y * scale_factor + offset_y)
            scaled_points.append([scaled_x, scaled_y])
        return np.array(scaled_points, np.int32)

    # スケーリング係数とオフセット
    scale_factor = 0.5
    offset_x = 100
    offset_y = 100

    # スケーリングと中央に寄せたセグメントを保持する辞書
    scaled_segments = {segment: scale_and_center_segment(points, scale_factor, offset_x, offset_y) for segment, points in segments.items()}

    # マウスコールバック関数
    def shape_selection(event, x, y, flags, param):
        global ref_point, cropping, moving, selected_segment, mode, image, scaled_segments

        if mode == 'select':
            if event == cv2.EVENT_LBUTTONDOWN:
                ref_point = [(x, y)]
                cropping = True
                selected_segment = None

            elif event == cv2.EVENT_MOUSEMOVE:
                if cropping:
                    temp_image = image.copy()
                    for points in scaled_segments.values():
                        cv2.polylines(temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if cropping:
                    ref_point.append((x, y))
                    cropping = False
                    for segment, points in scaled_segments.items():
                        if cv2.pointPolygonTest(points, ref_point[0], False) >= 0:
                            selected_segment = segment
                            break
                    image = original_image.copy()  # 元の画像に戻す
                    for points in scaled_segments.values():
                        cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    if selected_segment:
                        cv2.polylines(image, [scaled_segments[selected_segment]], isClosed=True, color=(0, 0, 255), thickness=2)
                    cv2.imshow("image", image)

        elif mode == 'move':
            if event == cv2.EVENT_LBUTTONDOWN:
                if selected_segment and cv2.pointPolygonTest(scaled_segments[selected_segment], (x, y), False) >= 0:
                    moving = True
                    ref_point = [(x, y)]
                    image = original_image.copy()  # 元の画像に戻す

            elif event == cv2.EVENT_MOUSEMOVE:
                if moving:
                    dx = x - ref_point[0][0]
                    dy = y - ref_point[0][1]
                    temp_image = original_image.copy()  # 元の画像に戻す
                    new_segments = {segment: points + [dx, dy] for segment, points in scaled_segments.items()}
                    for points in new_segments.values():
                        cv2.polylines(temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if moving:
                    dx = x - ref_point[0][0]
                    dy = y - ref_point[0][1]
                    scaled_segments = {segment: points + [dx, dy] for segment, points in scaled_segments.items()}
                    moving = False
                    image = original_image.copy()  # 元の画像に戻す
                    for points in scaled_segments.values():
                        cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    cv2.imshow("image", image)

    # ウィンドウを作成し、マウスコールバックを設定
    cv2.namedWindow("image")
    cv2.setMouseCallback("image", shape_selection)

    try:
        # 画像を表示
        while True:
            cv2.imshow("image", image)
            key = cv2.waitKey(1) & 0xFF

            if key == 27:  # ESCキーで終了
                break
            elif key == ord('s'):  # 's'キーで選択モードに切り替え
                mode = 'select'
                print("選択モード")
            elif key == ord('m'):  # 'm'キーで変形モードに切り替え
                mode = 'move'
                print("変形モード")

        if selected_segment:
            print(f"選択されたセグメント: {selected_segment}")

    except KeyboardInterrupt:
        print("プログラムが停止されました。")

    cv2.destroyAllWindows()

変形モード
変形モード
選択モード
変形モード
変形モード
選択モード
変形モード
選択されたセグメント: F


## rev19 各セグメントの描画領域を取得

In [89]:
import cv2
import numpy as np

# 画像の読み込み
image_path = 'segment_test.png'
image = cv2.imread(image_path)

# 画像が正しく読み込まれたか確認
if image is None:
    print(f"Error: 画像が見つかりません。パスを確認してください: {image_path}")
else:
    # グローバル変数
    ref_point = []
    cropping = False
    moving = False
    selected_segment = None
    mode = 'select'  # 初期モードは選択モード
    original_image = image.copy()  # 元の画像を保持

    # 7セグメントディスプレイの各セグメントの座標を定義
    segments = {
        'A': np.array([[50, 40], [140, 40], [160, 60], [140, 80], [50, 80], [30, 60]], np.int32),
        'B': np.array([[160, 60], [180, 80], [180, 180], [160, 200], [140, 180], [140, 80]], np.int32),
        'C': np.array([[160, 200], [180, 220], [180, 320], [160, 340], [140, 320], [140, 220]], np.int32),
        'D': np.array([[50, 320], [140, 320], [160, 340], [140, 360], [50, 360], [30, 340]], np.int32),
        'E': np.array([[30, 200], [50, 220], [50, 320], [30, 340], [10, 320], [10, 220]], np.int32),
        'F': np.array([[30, 60], [50, 80], [50, 180], [30, 200], [10, 180], [10, 80]], np.int32),
        'G': np.array([[50, 180], [140, 180], [160, 200], [140, 220], [50, 220], [30, 200]], np.int32),
        'DP': np.array([[180, 360], [200, 360], [200, 380], [180, 380]], np.int32)  # 小数点のドット
    }

    # スケーリングと中央に寄せるための変換行列を定義
    def scale_and_center_segment(points, scale_factor, offset_x, offset_y):
        scaled_points = []
        for x, y in points:
            scaled_x = int(x * scale_factor + offset_x)
            scaled_y = int(y * scale_factor + offset_y)
            scaled_points.append([scaled_x, scaled_y])
        return np.array(scaled_points, np.int32)

    # スケーリング係数とオフセット
    scale_factor = 0.5
    offset_x = 100
    offset_y = 100

    # スケーリングと中央に寄せたセグメントを保持する辞書
    scaled_segments = {segment: scale_and_center_segment(points, scale_factor, offset_x, offset_y) for segment, points in segments.items()}

    # 現在のセグメントの座標を取得する関数
    def get_current_segment_coordinates():
        return {segment: points.tolist() for segment, points in scaled_segments.items()}

    # マウスコールバック関数
    def shape_selection(event, x, y, flags, param):
        global ref_point, cropping, moving, selected_segment, mode, image, scaled_segments

        if mode == 'select':
            if event == cv2.EVENT_LBUTTONDOWN:
                ref_point = [(x, y)]
                cropping = True
                selected_segment = None

            elif event == cv2.EVENT_MOUSEMOVE:
                if cropping:
                    temp_image = image.copy()
                    for points in scaled_segments.values():
                        cv2.polylines(temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if cropping:
                    ref_point.append((x, y))
                    cropping = False
                    for segment, points in scaled_segments.items():
                        if cv2.pointPolygonTest(points, ref_point[0], False) >= 0:
                            selected_segment = segment
                            break
                    image = original_image.copy()  # 元の画像に戻す
                    for points in scaled_segments.values():
                        cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    if selected_segment:
                        cv2.polylines(image, [scaled_segments[selected_segment]], isClosed=True, color=(0, 0, 255), thickness=2)
                    cv2.imshow("image", image)

        elif mode == 'move':
            if event == cv2.EVENT_LBUTTONDOWN:
                if selected_segment and cv2.pointPolygonTest(scaled_segments[selected_segment], (x, y), False) >= 0:
                    moving = True
                    ref_point = [(x, y)]
                    image = original_image.copy()  # 元の画像に戻す

            elif event == cv2.EVENT_MOUSEMOVE:
                if moving:
                    dx = x - ref_point[0][0]
                    dy = y - ref_point[0][1]
                    temp_image = original_image.copy()  # 元の画像に戻す
                    new_segments = {segment: points + [dx, dy] for segment, points in scaled_segments.items()}
                    for points in new_segments.values():
                        cv2.polylines(temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if moving:
                    dx = x - ref_point[0][0]
                    dy = y - ref_point[0][1]
                    scaled_segments = {segment: points + [dx, dy] for segment, points in scaled_segments.items()}
                    moving = False
                    image = original_image.copy()  # 元の画像に戻す
                    for points in scaled_segments.values():
                        cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    cv2.imshow("image", image)

    # ウィンドウを作成し、マウスコールバックを設定
    cv2.namedWindow("image")
    cv2.setMouseCallback("image", shape_selection)

    try:
        # 画像を表示
        while True:
            cv2.imshow("image", image)
            key = cv2.waitKey(1) & 0xFF

            if key == 27:  # ESCキーで終了
                break
            elif key == ord('s'):  # 's'キーで選択モードに切り替え
                mode = 'select'
                print("選択モード")
            elif key == ord('m'):  # 'm'キーで移動モードに切り替え
                mode = 'move'
                print("移動モード")
            elif key == ord('c'):  # 'c'キーで現在のセグメントの座標を表示
                current_coordinates = get_current_segment_coordinates()
                print("現在のセグメントの座標:")
                for segment, coords in current_coordinates.items():
                    print(f"{segment}: {coords}")

        if selected_segment:
            print(f"選択されたセグメント: {selected_segment}")

    except KeyboardInterrupt:
        print("プログラムが停止されました。")

    cv2.destroyAllWindows()

## rev20　変形モード追加


In [91]:
import cv2
import numpy as np

# 画像の読み込み
image_path = 'segment_test.png'
image = cv2.imread(image_path)

# 画像が正しく読み込まれたか確認
if image is None:
    print(f"Error: 画像が見つかりません。パスを確認してください: {image_path}")
else:
    # グローバル変数
    ref_point = []
    cropping = False
    moving = False
    transforming = False
    selected_segment = None
    mode = 'select'  # 初期モードは選択モード
    original_image = image.copy()  # 元の画像を保持

    # 7セグメントディスプレイの各セグメントの座標を定義
    segments = {
        'A': np.array([[50, 40], [140, 40], [160, 60], [140, 80], [50, 80], [30, 60]], np.int32),
        'B': np.array([[160, 60], [180, 80], [180, 180], [160, 200], [140, 180], [140, 80]], np.int32),
        'C': np.array([[160, 200], [180, 220], [180, 320], [160, 340], [140, 320], [140, 220]], np.int32),
        'D': np.array([[50, 320], [140, 320], [160, 340], [140, 360], [50, 360], [30, 340]], np.int32),
        'E': np.array([[30, 200], [50, 220], [50, 320], [30, 340], [10, 320], [10, 220]], np.int32),
        'F': np.array([[30, 60], [50, 80], [50, 180], [30, 200], [10, 180], [10, 80]], np.int32),
        'G': np.array([[50, 180], [140, 180], [160, 200], [140, 220], [50, 220], [30, 200]], np.int32),
        'DP': np.array([[180, 360], [200, 360], [200, 380], [180, 380]], np.int32)  # 小数点のドット
    }

    # スケーリングと中央に寄せるための変換行列を定義
    def scale_and_center_segment(points, scale_factor, offset_x, offset_y):
        scaled_points = []
        for x, y in points:
            scaled_x = int(x * scale_factor + offset_x)
            scaled_y = int(y * scale_factor + offset_y)
            scaled_points.append([scaled_x, scaled_y])
        return np.array(scaled_points, np.int32)

    # スケーリング係数とオフセット
    scale_factor = 0.5
    offset_x = 100
    offset_y = 100

    # スケーリングと中央に寄せたセグメントを保持する辞書
    scaled_segments = {segment: scale_and_center_segment(points, scale_factor, offset_x, offset_y) for segment, points in segments.items()}

    # 現在のセグメントの座標を取得する関数
    def get_current_segment_coordinates():
        return {segment: points.tolist() for segment, points in scaled_segments.items()}

    # セグメントGを基準に他のセグメントを再計算する関数
    def recalculate_segments(scale_factor):
        g_points = segments['G']
        g_center = np.mean(g_points, axis=0)
        new_segments = {}
        for segment, points in segments.items():
            if segment == 'G':
                new_segments[segment] = scale_and_center_segment(points, scale_factor, offset_x, offset_y)
            else:
                relative_points = points - g_center
                scaled_points = relative_points * scale_factor + g_center
                new_segments[segment] = scale_and_center_segment(scaled_points, 1, offset_x, offset_y)
        return new_segments

    # マウスコールバック関数
    def shape_selection(event, x, y, flags, param):
        global ref_point, cropping, moving, transforming, selected_segment, mode, image, scaled_segments

        if mode == 'select':
            if event == cv2.EVENT_LBUTTONDOWN:
                ref_point = [(x, y)]
                cropping = True
                selected_segment = None

            elif event == cv2.EVENT_MOUSEMOVE:
                if cropping:
                    temp_image = image.copy()
                    for points in scaled_segments.values():
                        cv2.polylines(temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if cropping:
                    ref_point.append((x, y))
                    cropping = False
                    for segment, points in scaled_segments.items():
                        if cv2.pointPolygonTest(points, ref_point[0], False) >= 0:
                            selected_segment = segment
                            break
                    image = original_image.copy()  # 元の画像に戻す
                    for points in scaled_segments.values():
                        cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    if selected_segment:
                        cv2.polylines(image, [scaled_segments[selected_segment]], isClosed=True, color=(0, 0, 255), thickness=2)
                    cv2.imshow("image", image)

        elif mode == 'move':
            if event == cv2.EVENT_LBUTTONDOWN:
                if selected_segment and cv2.pointPolygonTest(scaled_segments[selected_segment], (x, y), False) >= 0:
                    moving = True
                    ref_point = [(x, y)]
                    image = original_image.copy()  # 元の画像に戻す

            elif event == cv2.EVENT_MOUSEMOVE:
                if moving:
                    dx = x - ref_point[0][0]
                    dy = y - ref_point[0][1]
                    temp_image = original_image.copy()  # 元の画像に戻す
                    new_segments = {segment: points + [dx, dy] for segment, points in scaled_segments.items()}
                    for points in new_segments.values():
                        cv2.polylines(temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if moving:
                    dx = x - ref_point[0][0]
                    dy = y - ref_point[0][1]
                    scaled_segments = {segment: points + [dx, dy] for segment, points in scaled_segments.items()}
                    moving = False
                    image = original_image.copy()  # 元の画像に戻す
                    for points in scaled_segments.values():
                        cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    cv2.imshow("image", image)

        elif mode == 'transform':
            if event == cv2.EVENT_LBUTTONDOWN:
                transforming = True
                ref_point = [(x, y)]
                image = original_image.copy()  # 元の画像に戻す

            elif event == cv2.EVENT_MOUSEMOVE:
                if transforming:
                    dx = x - ref_point[0][0]
                    scale_factor = 1 + dx / 100.0
                    temp_image = original_image.copy()  # 元の画像に戻す
                    new_segments = recalculate_segments(scale_factor)
                    for points in new_segments.values():
                        cv2.polylines(temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if transforming:
                    dx = x - ref_point[0][0]
                    scale_factor = 1 + dx / 100.0
                    scaled_segments = recalculate_segments(scale_factor)
                    transforming = False
                    image = original_image.copy()  # 元の画像に戻す
                    for points in scaled_segments.values():
                        cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    cv2.imshow("image", image)

    # ウィンドウを作成し、マウスコールバックを設定
    cv2.namedWindow("image")
    cv2.setMouseCallback("image", shape_selection)

    try:
        # 画像を表示
        while True:
            cv2.imshow("image", image)
            key = cv2.waitKey(1) & 0xFF

            if key == 27:  # ESCキーで終了
                break
            elif key == ord('s'):  # 's'キーで選択モードに切り替え
                mode = 'select'
                print("選択モード")
            elif key == ord('m'):  # 'm'キーで移動モードに切り替え
                mode = 'move'
                print("移動モード")
            elif key == ord('t'):  # 't'キーで変形モードに切り替え
                mode = 'transform'
                print("変形モード")
            elif key == ord('c'):  # 'c'キーで現在のセグメントの座標を表示
                current_coordinates = get_current_segment_coordinates()
                print("現在のセグメントの座標:")
                for segment, coords in current_coordinates.items():
                    print(f"{segment}: {coords}")

        if selected_segment:
            print(f"選択されたセグメント: {selected_segment}")

    except KeyboardInterrupt:
        print("プログラムが停止されました。")

    cv2.destroyAllWindows()

選択モード
現在のセグメントの座標:
A: [[125, 120], [170, 120], [180, 130], [170, 140], [125, 140], [115, 130]]
B: [[180, 130], [190, 140], [190, 190], [180, 200], [170, 190], [170, 140]]
C: [[180, 200], [190, 210], [190, 260], [180, 270], [170, 260], [170, 210]]
D: [[125, 260], [170, 260], [180, 270], [170, 280], [125, 280], [115, 270]]
E: [[115, 200], [125, 210], [125, 260], [115, 270], [105, 260], [105, 210]]
F: [[115, 130], [125, 140], [125, 190], [115, 200], [105, 190], [105, 140]]
G: [[125, 190], [170, 190], [180, 200], [170, 210], [125, 210], [115, 200]]
DP: [[190, 280], [200, 280], [200, 290], [190, 290]]
移動モード
変形モード
選択モード
移動モード
変形モード
移動モード
現在のセグメントの座標:
A: [[108, -7], [239, -7], [268, 22], [239, 51], [108, 51], [79, 22]]
B: [[268, 22], [298, 51], [298, 197], [268, 227], [239, 197], [239, 51]]
C: [[268, 227], [298, 256], [298, 402], [268, 431], [239, 402], [239, 256]]
D: [[108, 402], [239, 402], [268, 431], [239, 460], [108, 460], [79, 431]]
E: [[79, 227], [108, 256], [108, 402], [79, 431], [49,

## rev21 セグメントGの挙動修正

In [94]:
import cv2
import numpy as np

# 画像の読み込み
image_path = 'segment_test.png'
image = cv2.imread(image_path)

# 画像が正しく読み込まれたか確認
if image is None:
    print(f"Error: 画像が見つかりません。パスを確認してください: {image_path}")
else:
    # グローバル変数
    ref_point = []
    cropping = False
    moving = False
    transforming = False
    selected_segment = None
    mode = 'select'  # 初期モードは選択モード
    original_image = image.copy()  # 元の画像を保持

    # 7セグメントディスプレイの各セグメントの座標を定義
    segments = {
        'A': np.array([[50, 40], [140, 40], [160, 60], [140, 80], [50, 80], [30, 60]], np.int32),
        'B': np.array([[160, 60], [180, 80], [180, 180], [160, 200], [140, 180], [140, 80]], np.int32),
        'C': np.array([[160, 200], [180, 220], [180, 320], [160, 340], [140, 320], [140, 220]], np.int32),
        'D': np.array([[50, 320], [140, 320], [160, 340], [140, 360], [50, 360], [30, 340]], np.int32),
        'E': np.array([[30, 200], [50, 220], [50, 320], [30, 340], [10, 320], [10, 220]], np.int32),
        'F': np.array([[30, 60], [50, 80], [50, 180], [30, 200], [10, 180], [10, 80]], np.int32),
        'G': np.array([[50, 180], [140, 180], [160, 200], [140, 220], [50, 220], [30, 200]], np.int32),
        'DP': np.array([[180, 360], [200, 360], [200, 380], [180, 380]], np.int32)  # 小数点のドット
    }

    # スケーリングと中央に寄せるための変換行列を定義
    def scale_and_center_segment(points, scale_factor, offset_x, offset_y):
        scaled_points = []
        for x, y in points:
            scaled_x = int(x * scale_factor + offset_x)
            scaled_y = int(y * scale_factor + offset_y)
            scaled_points.append([scaled_x, scaled_y])
        return np.array(scaled_points, np.int32)

    # スケーリング係数とオフセット
    scale_factor = 0.5
    offset_x = 100
    offset_y = 100

    # スケーリングと中央に寄せたセグメントを保持する辞書
    scaled_segments = {segment: scale_and_center_segment(points, scale_factor, offset_x, offset_y) for segment, points in segments.items()}

    # 現在のセグメントの座標を取得する関数
    def get_current_segment_coordinates():
        return {segment: points.tolist() for segment, points in scaled_segments.items()}

    # セグメントGを基準に他のセグメントを再計算する関数
    def recalculate_segments(scale_factor):
        g_points = segments['G']
        g_center = np.mean(g_points, axis=0)
        new_segments = {}
        for segment, points in segments.items():
            if segment == 'G':
                scaled_points = (points - g_center) * scale_factor + g_center
                new_segments[segment] = scale_and_center_segment(scaled_points, 1, offset_x, offset_y)
            else:
                relative_points = points - g_center
                scaled_points = relative_points * scale_factor + g_center
                new_segments[segment] = scale_and_center_segment(scaled_points, 1, offset_x, offset_y)
        return new_segments

    # マウスコールバック関数
    def shape_selection(event, x, y, flags, param):
        global ref_point, cropping, moving, transforming, selected_segment, mode, image, scaled_segments

        if mode == 'select':
            if event == cv2.EVENT_LBUTTONDOWN:
                ref_point = [(x, y)]
                cropping = True
                selected_segment = None

            elif event == cv2.EVENT_MOUSEMOVE:
                if cropping:
                    temp_image = image.copy()
                    for points in scaled_segments.values():
                        cv2.polylines(temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if cropping:
                    ref_point.append((x, y))
                    cropping = False
                    for segment, points in scaled_segments.items():
                        if cv2.pointPolygonTest(points, ref_point[0], False) >= 0:
                            selected_segment = segment
                            break
                    image = original_image.copy()  # 元の画像に戻す
                    for points in scaled_segments.values():
                        cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    if selected_segment:
                        cv2.polylines(image, [scaled_segments[selected_segment]], isClosed=True, color=(0, 0, 255), thickness=2)
                    cv2.imshow("image", image)

        elif mode == 'move':
            if event == cv2.EVENT_LBUTTONDOWN:
                if selected_segment and cv2.pointPolygonTest(scaled_segments[selected_segment], (x, y), False) >= 0:
                    moving = True
                    ref_point = [(x, y)]
                    image = original_image.copy()  # 元の画像に戻す

            elif event == cv2.EVENT_MOUSEMOVE:
                if moving:
                    dx = x - ref_point[0][0]
                    dy = y - ref_point[0][1]
                    temp_image = original_image.copy()  # 元の画像に戻す
                    new_segments = {segment: points + [dx, dy] for segment, points in scaled_segments.items()}
                    for points in new_segments.values():
                        cv2.polylines(temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if moving:
                    dx = x - ref_point[0][0]
                    dy = y - ref_point[0][1]
                    scaled_segments = {segment: points + [dx, dy] for segment, points in scaled_segments.items()}
                    moving = False
                    image = original_image.copy()  # 元の画像に戻す
                    for points in scaled_segments.values():
                        cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    cv2.imshow("image", image)

        elif mode == 'transform':
            if event == cv2.EVENT_LBUTTONDOWN:
                transforming = True
                ref_point = [(x, y)]
                image = original_image.copy()  # 元の画像に戻す

            elif event == cv2.EVENT_MOUSEMOVE:
                if transforming:
                    dx = x - ref_point[0][0]
                    scale_factor = 1 + dx / 100.0
                    temp_image = original_image.copy()  # 元の画像に戻す
                    new_segments = recalculate_segments(scale_factor)
                    for points in new_segments.values():
                        cv2.polylines(temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if transforming:
                    dx = x - ref_point[0][0]
                    scale_factor = 1 + dx / 100.0
                    scaled_segments = recalculate_segments(scale_factor)
                    transforming = False
                    image = original_image.copy()  # 元の画像に戻す
                    for points in scaled_segments.values():
                        cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    cv2.imshow("image", image)

    # ウィンドウを作成し、マウスコールバックを設定
    cv2.namedWindow("image")
    cv2.setMouseCallback("image", shape_selection)

    try:
        # 画像を表示
        while True:
            cv2.imshow("image", image)
            key = cv2.waitKey(1) & 0xFF

            if key == 27:  # ESCキーで終了
                break
            elif key == ord('s'):  # 's'キーで選択モードに切り替え
                mode = 'select'
                print("選択モード")
            elif key == ord('m'):  # 'm'キーで移動モードに切り替え
                mode = 'move'
                print("移動モード")
            elif key == ord('t'):  # 't'キーで変形モードに切り替え
                mode = 'transform'
                print("変形モード")
            elif key == ord('c'):  # 'c'キーで現在のセグメントの座標を表示
                current_coordinates = get_current_segment_coordinates()
                print("現在のセグメントの座標:")
                for segment, coords in current_coordinates.items():
                    print(f"{segment}: {coords}")

        if selected_segment:
            print(f"選択されたセグメント: {selected_segment}")

    except KeyboardInterrupt:
        print("プログラムが停止されました。")

    cv2.destroyAllWindows()

選択モード
移動モード
変形モード
移動モード
選択モード
移動モード
移動モード
選択モード
移動モード
変形モード
移動モード
変形モード
移動モード
変形モード
変形モード
移動モード
変形モード
移動モード
移動モード
移動モード
選択モード
移動モード
移動モード
選択モード
移動モード
移動モード
現在のセグメントの座標:
A: [[211, 202], [222, 202], [225, 204], [222, 207], [211, 207], [208, 204]]
B: [[225, 204], [228, 207], [228, 220], [225, 223], [222, 220], [222, 207]]
C: [[225, 223], [228, 225], [228, 238], [225, 241], [222, 238], [222, 225]]
D: [[211, 238], [222, 238], [225, 241], [222, 243], [211, 243], [208, 241]]
E: [[208, 223], [211, 225], [211, 238], [208, 241], [205, 238], [205, 225]]
F: [[208, 204], [211, 207], [211, 220], [208, 223], [205, 220], [205, 207]]
G: [[211, 220], [222, 220], [225, 223], [222, 225], [211, 225], [208, 223]]
DP: [[228, 243], [230, 243], [230, 246], [228, 246]]
選択されたセグメント: D


## rev22 補助線追加

In [98]:
import cv2
import numpy as np

# 画像の読み込み
image_path = 'segment_test.png'
image = cv2.imread(image_path)

# 画像が正しく読み込まれたか確認
if image is None:
    print(f"Error: 画像が見つかりません。パスを確認してください: {image_path}")
else:
    # グローバル変数
    ref_point = []
    cropping = False
    moving = False
    transforming = False
    selected_segment = None
    mode = 'select'  # 初期モードは選択モード
    original_image = image.copy()  # 元の画像を保持

    # 7セグメントディスプレイの各セグメントの座標を定義
    segments = {
        'A': np.array([[50, 40], [140, 40], [160, 60], [140, 80], [50, 80], [30, 60]], np.int32),
        'B': np.array([[160, 60], [180, 80], [180, 180], [160, 200], [140, 180], [140, 80]], np.int32),
        'C': np.array([[160, 200], [180, 220], [180, 320], [160, 340], [140, 320], [140, 220]], np.int32),
        'D': np.array([[50, 320], [140, 320], [160, 340], [140, 360], [50, 360], [30, 340]], np.int32),
        'E': np.array([[30, 200], [50, 220], [50, 320], [30, 340], [10, 320], [10, 220]], np.int32),
        'F': np.array([[30, 60], [50, 80], [50, 180], [30, 200], [10, 180], [10, 80]], np.int32),
        'G': np.array([[50, 180], [140, 180], [160, 200], [140, 220], [50, 220], [30, 200]], np.int32),
        'DP': np.array([[180, 360], [200, 360], [200, 380], [180, 380]], np.int32)  # 小数点のドット
    }

    # スケーリングと中央に寄せるための変換行列を定義
    def scale_and_center_segment(points, scale_factor, offset_x, offset_y):
        scaled_points = []
        for x, y in points:
            scaled_x = int(x * scale_factor + offset_x)
            scaled_y = int(y * scale_factor + offset_y)
            scaled_points.append([scaled_x, scaled_y])
        return np.array(scaled_points, np.int32)

    # スケーリング係数とオフセット
    scale_factor = 0.5
    offset_x = 100
    offset_y = 100

    # スケーリングと中央に寄せたセグメントを保持する辞書
    scaled_segments = {segment: scale_and_center_segment(points, scale_factor, offset_x, offset_y) for segment, points in segments.items()}

    # 現在のセグメントの座標を取得する関数
    def get_current_segment_coordinates():
        return {segment: points.tolist() for segment, points in scaled_segments.items()}

    # セグメントGを基準に他のセグメントを再計算する関数
    def recalculate_segments(scale_factor):
        g_points = segments['G']
        g_center = np.mean(g_points, axis=0)
        new_segments = {}
        for segment, points in segments.items():
            if segment == 'G':
                scaled_points = (points - g_center) * scale_factor + g_center
                new_segments[segment] = scale_and_center_segment(scaled_points, 1, offset_x, offset_y)
            else:
                relative_points = points - g_center
                scaled_points = relative_points * scale_factor + g_center
                new_segments[segment] = scale_and_center_segment(scaled_points, 1, offset_x, offset_y)
        return new_segments

    # 7セグメントを囲う補助線を描画する関数
    def draw_bounding_box(image, segments):
        all_points = np.concatenate(list(segments.values()))
        x, y, w, h = cv2.boundingRect(all_points)
        cv2.rectangle(image, (x, y), (x + w, y + h), (255, 255, 255), 1, lineType=cv2.LINE_AA)
        return (x, y, x + w, y + h)

    # マウスコールバック関数
    def shape_selection(event, x, y, flags, param):
        global ref_point, cropping, moving, transforming, selected_segment, mode, image, scaled_segments

        if mode == 'select':
            if event == cv2.EVENT_LBUTTONDOWN:
                ref_point = [(x, y)]
                cropping = True
                selected_segment = None

            elif event == cv2.EVENT_MOUSEMOVE:
                if cropping:
                    temp_image = image.copy()
                    for points in scaled_segments.values():
                        cv2.polylines(temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    draw_bounding_box(temp_image, scaled_segments)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if cropping:
                    ref_point.append((x, y))
                    cropping = False
                    for segment, points in scaled_segments.items():
                        if cv2.pointPolygonTest(points, ref_point[0], False) >= 0:
                            selected_segment = segment
                            break
                    image = original_image.copy()  # 元の画像に戻す
                    for points in scaled_segments.values():
                        cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    if selected_segment:
                        cv2.polylines(image, [scaled_segments[selected_segment]], isClosed=True, color=(0, 0, 255), thickness=2)
                    draw_bounding_box(image, scaled_segments)
                    cv2.imshow("image", image)

        elif mode == 'move':
            if event == cv2.EVENT_LBUTTONDOWN:
                if selected_segment and cv2.pointPolygonTest(scaled_segments[selected_segment], (x, y), False) >= 0:
                    moving = True
                    ref_point = [(x, y)]
                    image = original_image.copy()  # 元の画像に戻す

            elif event == cv2.EVENT_MOUSEMOVE:
                if moving:
                    dx = x - ref_point[0][0]
                    dy = y - ref_point[0][1]
                    temp_image = original_image.copy()  # 元の画像に戻す
                    new_segments = {segment: points + [dx, dy] for segment, points in scaled_segments.items()}
                    for points in new_segments.values():
                        cv2.polylines(temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    draw_bounding_box(temp_image, new_segments)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if moving:
                    dx = x - ref_point[0][0]
                    dy = y - ref_point[0][1]
                    scaled_segments = {segment: points + [dx, dy] for segment, points in scaled_segments.items()}
                    moving = False
                    image = original_image.copy()  # 元の画像に戻す
                    for points in scaled_segments.values():
                        cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    draw_bounding_box(image, scaled_segments)
                    cv2.imshow("image", image)

        elif mode == 'transform':
            if event == cv2.EVENT_LBUTTONDOWN:
                transforming = True
                ref_point = [(x, y)]
                image = original_image.copy()  # 元の画像に戻す

            elif event == cv2.EVENT_MOUSEMOVE:
                if transforming:
                    dx = x - ref_point[0][0]
                    scale_factor = 1 + dx / 100.0
                    temp_image = original_image.copy()  # 元の画像に戻す
                    new_segments = recalculate_segments(scale_factor)
                    for points in new_segments.values():
                        cv2.polylines(temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    draw_bounding_box(temp_image, new_segments)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if transforming:
                    dx = x - ref_point[0][0]
                    scale_factor = 1 + dx / 100.0
                    scaled_segments = recalculate_segments(scale_factor)
                    transforming = False
                    image = original_image.copy()  # 元の画像に戻す
                    for points in scaled_segments.values():
                        cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    draw_bounding_box(image, scaled_segments)
                    cv2.imshow("image", image)

    # ウィンドウを作成し、マウスコールバックを設定
    cv2.namedWindow("image")
    cv2.setMouseCallback("image", shape_selection)

    try:
        # 画像を表示
        while True:
            cv2.imshow("image", image)
            key = cv2.waitKey(1) & 0xFF

            if key == 27:  # ESCキーで終了
                break
            elif key == ord('s'):  # 's'キーで選択モードに切り替え
                mode = 'select'
                print("選択モード")
            elif key == ord('m'):  # 'm'キーで移動モードに切り替え
                mode = 'move'
                print("移動モード")
            elif key == ord('t'):  # 't'キーで変形モードに切り替え
                mode = 'transform'
                print("変形モード")
            elif key == ord('c'):  # 'c'キーで現在のセグメントの座標を表示
                current_coordinates = get_current_segment_coordinates()
                print("現在のセグメントの座標:")
                for segment, coords in current_coordinates.items():
                    print(f"{segment}: {coords}")

        if selected_segment:
            print(f"選択されたセグメント: {selected_segment}")

    except KeyboardInterrupt:
        print("プログラムが停止されました。")

    cv2.destroyAllWindows()

## rev23 拡大縮小を補助線ベースで行う


In [99]:
import cv2
import numpy as np

# 画像の読み込み
image_path = 'segment_test.png'
image = cv2.imread(image_path)

# 画像が正しく読み込まれたか確認
if image is None:
    print(f"Error: 画像が見つかりません。パスを確認してください: {image_path}")
else:
    # グローバル変数
    ref_point = []
    cropping = False
    moving = False
    transforming = False
    selected_segment = None
    mode = 'select'  # 初期モードは選択モード
    original_image = image.copy()  # 元の画像を保持
    bounding_box = None

    # 7セグメントディスプレイの各セグメントの座標を定義
    segments = {
        'A': np.array([[50, 40], [140, 40], [160, 60], [140, 80], [50, 80], [30, 60]], np.int32),
        'B': np.array([[160, 60], [180, 80], [180, 180], [160, 200], [140, 180], [140, 80]], np.int32),
        'C': np.array([[160, 200], [180, 220], [180, 320], [160, 340], [140, 320], [140, 220]], np.int32),
        'D': np.array([[50, 320], [140, 320], [160, 340], [140, 360], [50, 360], [30, 340]], np.int32),
        'E': np.array([[30, 200], [50, 220], [50, 320], [30, 340], [10, 320], [10, 220]], np.int32),
        'F': np.array([[30, 60], [50, 80], [50, 180], [30, 200], [10, 180], [10, 80]], np.int32),
        'G': np.array([[50, 180], [140, 180], [160, 200], [140, 220], [50, 220], [30, 200]], np.int32),
        'DP': np.array([[180, 360], [200, 360], [200, 380], [180, 380]], np.int32)  # 小数点のドット
    }

    # スケーリングと中央に寄せるための変換行列を定義
    def scale_and_center_segment(points, scale_factor, offset_x, offset_y):
        scaled_points = []
        for x, y in points:
            scaled_x = int(x * scale_factor + offset_x)
            scaled_y = int(y * scale_factor + offset_y)
            scaled_points.append([scaled_x, scaled_y])
        return np.array(scaled_points, np.int32)

    # スケーリング係数とオフセット
    scale_factor = 0.5
    offset_x = 100
    offset_y = 100

    # スケーリングと中央に寄せたセグメントを保持する辞書
    scaled_segments = {segment: scale_and_center_segment(points, scale_factor, offset_x, offset_y) for segment, points in segments.items()}

    # 現在のセグメントの座標を取得する関数
    def get_current_segment_coordinates():
        return {segment: points.tolist() for segment, points in scaled_segments.items()}

    # セグメントGを基準に他のセグメントを再計算する関数
    def recalculate_segments(scale_factor):
        g_points = segments['G']
        g_center = np.mean(g_points, axis=0)
        new_segments = {}
        for segment, points in segments.items():
            if segment == 'G':
                scaled_points = (points - g_center) * scale_factor + g_center
                new_segments[segment] = scale_and_center_segment(scaled_points, 1, offset_x, offset_y)
            else:
                relative_points = points - g_center
                scaled_points = relative_points * scale_factor + g_center
                new_segments[segment] = scale_and_center_segment(scaled_points, 1, offset_x, offset_y)
        return new_segments

    # 7セグメントを囲う補助線を描画する関数
    def draw_bounding_box(image, segments):
        all_points = np.concatenate(list(segments.values()))
        x, y, w, h = cv2.boundingRect(all_points)
        cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 1, lineType=cv2.LINE_AA)
        return (x, y, x + w, y + h)

    # マウスコールバック関数
    def shape_selection(event, x, y, flags, param):
        global ref_point, cropping, moving, transforming, selected_segment, mode, image, scaled_segments, bounding_box

        if mode == 'select':
            if event == cv2.EVENT_LBUTTONDOWN:
                ref_point = [(x, y)]
                cropping = True
                selected_segment = None

            elif event == cv2.EVENT_MOUSEMOVE:
                if cropping:
                    temp_image = image.copy()
                    for points in scaled_segments.values():
                        cv2.polylines(temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    bounding_box = draw_bounding_box(temp_image, scaled_segments)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if cropping:
                    ref_point.append((x, y))
                    cropping = False
                    for segment, points in scaled_segments.items():
                        if cv2.pointPolygonTest(points, ref_point[0], False) >= 0:
                            selected_segment = segment
                            break
                    image = original_image.copy()  # 元の画像に戻す
                    for points in scaled_segments.values():
                        cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    if selected_segment:
                        cv2.polylines(image, [scaled_segments[selected_segment]], isClosed=True, color=(0, 0, 255), thickness=2)
                    bounding_box = draw_bounding_box(image, scaled_segments)
                    cv2.imshow("image", image)

        elif mode == 'move':
            if event == cv2.EVENT_LBUTTONDOWN:
                if bounding_box and bounding_box[0] <= x <= bounding_box[2] and bounding_box[1] <= y <= bounding_box[3]:
                    moving = True
                    ref_point = [(x, y)]
                    image = original_image.copy()  # 元の画像に戻す

            elif event == cv2.EVENT_MOUSEMOVE:
                if moving:
                    dx = x - ref_point[0][0]
                    dy = y - ref_point[0][1]
                    temp_image = original_image.copy()  # 元の画像に戻す
                    new_segments = {segment: points + [dx, dy] for segment, points in scaled_segments.items()}
                    for points in new_segments.values():
                        cv2.polylines(temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    bounding_box = draw_bounding_box(temp_image, new_segments)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if moving:
                    dx = x - ref_point[0][0]
                    dy = y - ref_point[0][1]
                    scaled_segments = {segment: points + [dx, dy] for segment, points in scaled_segments.items()}
                    moving = False
                    image = original_image.copy()  # 元の画像に戻す
                    for points in scaled_segments.values():
                        cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    bounding_box = draw_bounding_box(image, scaled_segments)
                    cv2.imshow("image", image)

        elif mode == 'transform':
            if event == cv2.EVENT_LBUTTONDOWN:
                if bounding_box and bounding_box[0] <= x <= bounding_box[2] and bounding_box[1] <= y <= bounding_box[3]:
                    transforming = True
                    ref_point = [(x, y)]
                    image = original_image.copy()  # 元の画像に戻す

            elif event == cv2.EVENT_MOUSEMOVE:
                if transforming:
                    dx = x - ref_point[0][0]
                    scale_factor = 1 + dx / 100.0
                    temp_image = original_image.copy()  # 元の画像に戻す
                    new_segments = recalculate_segments(scale_factor)
                    for points in new_segments.values():
                        cv2.polylines(temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    bounding_box = draw_bounding_box(temp_image, new_segments)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if transforming:
                    dx = x - ref_point[0][0]
                    scale_factor = 1 + dx / 100.0
                    scaled_segments = recalculate_segments(scale_factor)
                    transforming = False
                    image = original_image.copy()  # 元の画像に戻す
                    for points in scaled_segments.values():
                        cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    bounding_box = draw_bounding_box(image, scaled_segments)
                    cv2.imshow("image", image)

    # ウィンドウを作成し、マウスコールバックを設定
    cv2.namedWindow("image")
    cv2.setMouseCallback("image", shape_selection)

    try:
        # 画像を表示
        while True:
            cv2.imshow("image", image)
            key = cv2.waitKey(1) & 0xFF

            if key == 27:  # ESCキーで終了
                break
            elif key == ord('s'):  # 's'キーで選択モードに切り替え
                mode = 'select'
                print("選択モード")
            elif key == ord('m'):  # 'm'キーで移動モードに切り替え
                mode = 'move'
                print("移動モード")
            elif key == ord('t'):  # 't'キーで変形モードに切り替え
                mode = 'transform'
                print("変形モード")
            elif key == ord('c'):  # 'c'キーで現在のセグメントの座標を表示
                current_coordinates = get_current_segment_coordinates()
                print("現在のセグメントの座標:")
                for segment, coords in current_coordinates.items():
                    print(f"{segment}: {coords}")

        if selected_segment:
            print(f"選択されたセグメント: {selected_segment}")

    except KeyboardInterrupt:
        print("プログラムが停止されました。")

    cv2.destroyAllWindows()

移動モード
変形モード


## rev24 図形の変形斜めに

In [101]:
import cv2
import numpy as np

# 画像の読み込み
image_path = 'segment_test.png'
image = cv2.imread(image_path)

# 画像が正しく読み込まれたか確認
if image is None:
    print(f"Error: 画像が見つかりません。パスを確認してください: {image_path}")
else:
    # グローバル変数
    ref_point = []
    cropping = False
    moving = False
    transforming = False
    adjusting = False
    selected_segment = None
    mode = 'select'  # 初期モードは選択モード
    original_image = image.copy()  # 元の画像を保持
    bounding_box = None

    # 7セグメントディスプレイの各セグメントの座標を定義
    segments = {
        'A': np.array([[50, 40], [140, 40], [160, 60], [140, 80], [50, 80], [30, 60]], np.int32),
        'B': np.array([[160, 60], [180, 80], [180, 180], [160, 200], [140, 180], [140, 80]], np.int32),
        'C': np.array([[160, 200], [180, 220], [180, 320], [160, 340], [140, 320], [140, 220]], np.int32),
        'D': np.array([[50, 320], [140, 320], [160, 340], [140, 360], [50, 360], [30, 340]], np.int32),
        'E': np.array([[30, 200], [50, 220], [50, 320], [30, 340], [10, 320], [10, 220]], np.int32),
        'F': np.array([[30, 60], [50, 80], [50, 180], [30, 200], [10, 180], [10, 80]], np.int32),
        'G': np.array([[50, 180], [140, 180], [160, 200], [140, 220], [50, 220], [30, 200]], np.int32),
        'DP': np.array([[180, 360], [200, 360], [200, 380], [180, 380]], np.int32)  # 小数点のドット
    }

    # スケーリングと中央に寄せるための変換行列を定義
    def scale_and_center_segment(points, scale_factor, offset_x, offset_y):
        scaled_points = []
        for x, y in points:
            scaled_x = int(x * scale_factor + offset_x)
            scaled_y = int(y * scale_factor + offset_y)
            scaled_points.append([scaled_x, scaled_y])
        return np.array(scaled_points, np.int32)

    # スケーリング係数とオフセット
    scale_factor = 0.5
    offset_x = 100
    offset_y = 100

    # スケーリングと中央に寄せたセグメントを保持する辞書
    scaled_segments = {segment: scale_and_center_segment(points, scale_factor, offset_x, offset_y) for segment, points in segments.items()}

    # 現在のセグメントの座標を取得する関数
    def get_current_segment_coordinates():
        return {segment: points.tolist() for segment, points in scaled_segments.items()}

    # セグメントGを基準に他のセグメントを再計算する関数
    def recalculate_segments(scale_factor):
        g_points = segments['G']
        g_center = np.mean(g_points, axis=0)
        new_segments = {}
        for segment, points in segments.items():
            if segment == 'G':
                scaled_points = (points - g_center) * scale_factor + g_center
                new_segments[segment] = scale_and_center_segment(scaled_points, 1, offset_x, offset_y)
            else:
                relative_points = points - g_center
                scaled_points = relative_points * scale_factor + g_center
                new_segments[segment] = scale_and_center_segment(scaled_points, 1, offset_x, offset_y)
        return new_segments

    # 7セグメントを囲う補助線を描画する関数
    def draw_bounding_box(image, segments):
        all_points = np.concatenate(list(segments.values()))
        x, y, w, h = cv2.boundingRect(all_points)
        cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 1, lineType=cv2.LINE_AA)
        return (x, y, x + w, y + h)

    # セグメントの傾きを調整する関数
    def adjust_segment_tilt(points, angle):
        center = np.mean(points, axis=0)
        rotation_matrix = cv2.getRotationMatrix2D(tuple(center), angle, 1.0)
        adjusted_points = cv2.transform(np.array([points]), rotation_matrix)[0]
        return adjusted_points

    # マウスコールバック関数
    def shape_selection(event, x, y, flags, param):
        global ref_point, cropping, moving, transforming, adjusting, selected_segment, mode, image, scaled_segments, bounding_box

        if mode == 'select':
            if event == cv2.EVENT_LBUTTONDOWN:
                ref_point = [(x, y)]
                cropping = True
                selected_segment = None

            elif event == cv2.EVENT_MOUSEMOVE:
                if cropping:
                    temp_image = image.copy()
                    for points in scaled_segments.values():
                        cv2.polylines(temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    bounding_box = draw_bounding_box(temp_image, scaled_segments)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if cropping:
                    ref_point.append((x, y))
                    cropping = False
                    for segment, points in scaled_segments.items():
                        if cv2.pointPolygonTest(points, ref_point[0], False) >= 0:
                            selected_segment = segment
                            break
                    image = original_image.copy()  # 元の画像に戻す
                    for points in scaled_segments.values():
                        cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    if selected_segment:
                        cv2.polylines(image, [scaled_segments[selected_segment]], isClosed=True, color=(0, 0, 255), thickness=2)
                    bounding_box = draw_bounding_box(image, scaled_segments)
                    cv2.imshow("image", image)

        elif mode == 'move':
            if event == cv2.EVENT_LBUTTONDOWN:
                if bounding_box and bounding_box[0] <= x <= bounding_box[2] and bounding_box[1] <= y <= bounding_box[3]:
                    moving = True
                    ref_point = [(x, y)]
                    image = original_image.copy()  # 元の画像に戻す

            elif event == cv2.EVENT_MOUSEMOVE:
                if moving:
                    dx = x - ref_point[0][0]
                    dy = y - ref_point[0][1]
                    temp_image = original_image.copy()  # 元の画像に戻す
                    new_segments = {segment: points + [dx, dy] for segment, points in scaled_segments.items()}
                    for points in new_segments.values():
                        cv2.polylines(temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    bounding_box = draw_bounding_box(temp_image, new_segments)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if moving:
                    dx = x - ref_point[0][0]
                    dy = y - ref_point[0][1]
                    scaled_segments = {segment: points + [dx, dy] for segment, points in scaled_segments.items()}
                    moving = False
                    image = original_image.copy()  # 元の画像に戻す
                    for points in scaled_segments.values():
                        cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    bounding_box = draw_bounding_box(image, scaled_segments)
                    cv2.imshow("image", image)

        elif mode == 'transform':
            if event == cv2.EVENT_LBUTTONDOWN:
                if bounding_box and bounding_box[0] <= x <= bounding_box[2] and bounding_box[1] <= y <= bounding_box[3]:
                    transforming = True
                    ref_point = [(x, y)]
                    image = original_image.copy()  # 元の画像に戻す

            elif event == cv2.EVENT_MOUSEMOVE:
                if transforming:
                    dx = x - ref_point[0][0]
                    scale_factor = 1 + dx / 100.0
                    temp_image = original_image.copy()  # 元の画像に戻す
                    new_segments = recalculate_segments(scale_factor)
                    for points in new_segments.values():
                        cv2.polylines(temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    bounding_box = draw_bounding_box(temp_image, new_segments)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if transforming:
                    dx = x - ref_point[0][0]
                    scale_factor = 1 + dx / 100.0
                    scaled_segments = recalculate_segments(scale_factor)
                    transforming = False
                    image = original_image.copy()  # 元の画像に戻す
                    for points in scaled_segments.values():
                        cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    bounding_box = draw_bounding_box(image, scaled_segments)
                    cv2.imshow("image", image)

        elif mode == 'adjust':
            if event == cv2.EVENT_LBUTTONDOWN:
                if bounding_box and bounding_box[0] <= x <= bounding_box[2] and bounding_box[1] <= y <= bounding_box[3]:
                    adjusting = True
                    ref_point = [(x, y)]
                    image = original_image.copy()  # 元の画像に戻す

            elif event == cv2.EVENT_MOUSEMOVE:
                if adjusting:
                    dx = x - ref_point[0][0]
                    angle = dx / 5.0  # 角度を調整するためのスケール
                    temp_image = original_image.copy()  # 元の画像に戻す
                    new_segments = {segment: adjust_segment_tilt(points, angle) for segment, points in scaled_segments.items()}
                    for points in new_segments.values():
                        cv2.polylines(temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    bounding_box = draw_bounding_box(temp_image, new_segments)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if adjusting:
                    dx = x - ref_point[0][0]
                    angle = dx / 5.0  # 角度を調整するためのスケール
                    scaled_segments = {segment: adjust_segment_tilt(points, angle) for segment, points in scaled_segments.items()}
                    adjusting = False
                    image = original_image.copy()  # 元の画像に戻す
                    for points in scaled_segments.values():
                        cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    bounding_box = draw_bounding_box(image, scaled_segments)
                    cv2.imshow("image", image)

    # ウィンドウを作成し、マウスコールバックを設定
    cv2.namedWindow("image")
    cv2.setMouseCallback("image", shape_selection)

    try:
        # 画像を表示
        while True:
            cv2.imshow("image", image)
            key = cv2.waitKey(1) & 0xFF

            if key == 27:  # ESCキーで終了
                break
            elif key == ord('s'):  # 's'キーで選択モードに切り替え
                mode = 'select'
                print("選択モード")
            elif key == ord('m'):  # 'm'キーで移動モードに切り替え
                mode = 'move'
                print("移動モード")
            elif key == ord('t'):  # 't'キーで変形モードに切り替え
                mode = 'transform'
                print("変形モード")
            elif key == ord('a'):  # 'a'キーで図形補正モードに切り替え
                mode = 'adjust'
                print("図形補正モード")
            elif key == ord('c'):  # 'c'キーで現在のセグメントの座標を表示
                current_coordinates = get_current_segment_coordinates()
                print("現在のセグメントの座標:")
                for segment, coords in current_coordinates.items():
                    print(f"{segment}: {coords}")

        if selected_segment:
            print(f"選択されたセグメント: {selected_segment}")

    except KeyboardInterrupt:
        print("プログラムが停止されました。")

    cv2.destroyAllWindows()

移動モード
変形モード
図形補正モード
変形モード
移動モード
変形モード
移動モード
移動モード
移動モード
図形補正モード
移動モード
変形モード
移動モード
図形補正モード
変形モード
移動モード


## rev25 補正後のセグメントが飛ぶバグを修正

In [102]:
import cv2
import numpy as np

# 画像の読み込み
image_path = 'segment_test.png'
image = cv2.imread(image_path)

# 画像が正しく読み込まれたか確認
if image is None:
    print(f"Error: 画像が見つかりません。パスを確認してください: {image_path}")
else:
    # グローバル変数
    ref_point = []
    cropping = False
    moving = False
    transforming = False
    adjusting = False
    selected_segment = None
    mode = 'select'  # 初期モードは選択モード
    original_image = image.copy()  # 元の画像を保持
    bounding_box = None

    # 7セグメントディスプレイの各セグメントの座標を定義
    segments = {
        'A': np.array([[50, 40], [140, 40], [160, 60], [140, 80], [50, 80], [30, 60]], np.int32),
        'B': np.array([[160, 60], [180, 80], [180, 180], [160, 200], [140, 180], [140, 80]], np.int32),
        'C': np.array([[160, 200], [180, 220], [180, 320], [160, 340], [140, 320], [140, 220]], np.int32),
        'D': np.array([[50, 320], [140, 320], [160, 340], [140, 360], [50, 360], [30, 340]], np.int32),
        'E': np.array([[30, 200], [50, 220], [50, 320], [30, 340], [10, 320], [10, 220]], np.int32),
        'F': np.array([[30, 60], [50, 80], [50, 180], [30, 200], [10, 180], [10, 80]], np.int32),
        'G': np.array([[50, 180], [140, 180], [160, 200], [140, 220], [50, 220], [30, 200]], np.int32),
        'DP': np.array([[180, 360], [200, 360], [200, 380], [180, 380]], np.int32)  # 小数点のドット
    }

    # スケーリングと中央に寄せるための変換行列を定義
    def scale_and_center_segment(points, scale_factor, offset_x, offset_y):
        scaled_points = []
        for x, y in points:
            scaled_x = int(x * scale_factor + offset_x)
            scaled_y = int(y * scale_factor + offset_y)
            scaled_points.append([scaled_x, scaled_y])
        return np.array(scaled_points, np.int32)

    # スケーリング係数とオフセット
    scale_factor = 0.5
    offset_x = 100
    offset_y = 100

    # スケーリングと中央に寄せたセグメントを保持する辞書
    scaled_segments = {segment: scale_and_center_segment(points, scale_factor, offset_x, offset_y) for segment, points in segments.items()}

    # 現在のセグメントの座標を取得する関数
    def get_current_segment_coordinates():
        return {segment: points.tolist() for segment, points in scaled_segments.items()}

    # セグメントGを基準に他のセグメントを再計算する関数
    def recalculate_segments(scale_factor):
        g_points = segments['G']
        g_center = np.mean(g_points, axis=0)
        new_segments = {}
        for segment, points in segments.items():
            if segment == 'G':
                scaled_points = (points - g_center) * scale_factor + g_center
                new_segments[segment] = scale_and_center_segment(scaled_points, 1, offset_x, offset_y)
            else:
                relative_points = points - g_center
                scaled_points = relative_points * scale_factor + g_center
                new_segments[segment] = scale_and_center_segment(scaled_points, 1, offset_x, offset_y)
        return new_segments

    # 7セグメントを囲う補助線を描画する関数
    def draw_bounding_box(image, segments):
        all_points = np.concatenate(list(segments.values()))
        x, y, w, h = cv2.boundingRect(all_points)
        cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 1, lineType=cv2.LINE_AA)
        return (x, y, x + w, y + h)

    # セグメントの傾きを調整する関数
    def adjust_segment_tilt(points, angle):
        center = np.mean(points, axis=0)
        rotation_matrix = cv2.getRotationMatrix2D(tuple(center), angle, 1.0)
        adjusted_points = cv2.transform(np.array([points]), rotation_matrix)[0]
        return adjusted_points

    # 画像を更新する関数
    def update_image():
        global image, scaled_segments, bounding_box
        image = original_image.copy()  # 元の画像に戻す
        for points in scaled_segments.values():
            cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
        bounding_box = draw_bounding_box(image, scaled_segments)
        cv2.imshow("image", image)

    # マウスコールバック関数
    def shape_selection(event, x, y, flags, param):
        global ref_point, cropping, moving, transforming, adjusting, selected_segment, mode, image, scaled_segments, bounding_box

        if mode == 'select':
            if event == cv2.EVENT_LBUTTONDOWN:
                ref_point = [(x, y)]
                cropping = True
                selected_segment = None

            elif event == cv2.EVENT_MOUSEMOVE:
                if cropping:
                    temp_image = image.copy()
                    for points in scaled_segments.values():
                        cv2.polylines(temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    bounding_box = draw_bounding_box(temp_image, scaled_segments)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if cropping:
                    ref_point.append((x, y))
                    cropping = False
                    for segment, points in scaled_segments.items():
                        if cv2.pointPolygonTest(points, ref_point[0], False) >= 0:
                            selected_segment = segment
                            break
                    update_image()
                    if selected_segment:
                        cv2.polylines(image, [scaled_segments[selected_segment]], isClosed=True, color=(0, 0, 255), thickness=2)
                    cv2.imshow("image", image)

        elif mode == 'move':
            if event == cv2.EVENT_LBUTTONDOWN:
                if bounding_box and bounding_box[0] <= x <= bounding_box[2] and bounding_box[1] <= y <= bounding_box[3]:
                    moving = True
                    ref_point = [(x, y)]

            elif event == cv2.EVENT_MOUSEMOVE:
                if moving:
                    dx = x - ref_point[0][0]
                    dy = y - ref_point[0][1]
                    temp_image = original_image.copy()  # 元の画像に戻す
                    new_segments = {segment: points + [dx, dy] for segment, points in scaled_segments.items()}
                    for points in new_segments.values():
                        cv2.polylines(temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    bounding_box = draw_bounding_box(temp_image, new_segments)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if moving:
                    dx = x - ref_point[0][0]
                    dy = y - ref_point[0][1]
                    scaled_segments = {segment: points + [dx, dy] for segment, points in scaled_segments.items()}
                    moving = False
                    update_image()

        elif mode == 'transform':
            if event == cv2.EVENT_LBUTTONDOWN:
                if bounding_box and bounding_box[0] <= x <= bounding_box[2] and bounding_box[1] <= y <= bounding_box[3]:
                    transforming = True
                    ref_point = [(x, y)]

            elif event == cv2.EVENT_MOUSEMOVE:
                if transforming:
                    dx = x - ref_point[0][0]
                    scale_factor = 1 + dx / 100.0
                    temp_image = original_image.copy()  # 元の画像に戻す
                    new_segments = recalculate_segments(scale_factor)
                    for points in new_segments.values():
                        cv2.polylines(temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    bounding_box = draw_bounding_box(temp_image, new_segments)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if transforming:
                    dx = x - ref_point[0][0]
                    scale_factor = 1 + dx / 100.0
                    scaled_segments = recalculate_segments(scale_factor)
                    transforming = False
                    update_image()

        elif mode == 'adjust':
            if event == cv2.EVENT_LBUTTONDOWN:
                if bounding_box and bounding_box[0] <= x <= bounding_box[2] and bounding_box[1] <= y <= bounding_box[3]:
                    adjusting = True
                    ref_point = [(x, y)]

            elif event == cv2.EVENT_MOUSEMOVE:
                if adjusting:
                    dx = x - ref_point[0][0]
                    angle = dx / 5.0  # 角度を調整するためのスケール
                    temp_image = original_image.copy()  # 元の画像に戻す
                    new_segments = {segment: adjust_segment_tilt(points, angle) for segment, points in scaled_segments.items()}
                    for points in new_segments.values():
                        cv2.polylines(temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    bounding_box = draw_bounding_box(temp_image, new_segments)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if adjusting:
                    dx = x - ref_point[0][0]
                    angle = dx / 5.0  # 角度を調整するためのスケール
                    scaled_segments = {segment: adjust_segment_tilt(points, angle) for segment, points in scaled_segments.items()}
                    adjusting = False
                    update_image()

    # ウィンドウを作成し、マウスコールバックを設定
    cv2.namedWindow("image")
    cv2.setMouseCallback("image", shape_selection)

    try:
        # 画像を表示
        while True:
            cv2.imshow("image", image)
            key = cv2.waitKey(1) & 0xFF

            if key == 27:  # ESCキーで終了
                break
            elif key == ord('s'):  # 's'キーで選択モードに切り替え
                mode = 'select'
                print("選択モード")
                update_image()
            elif key == ord('m'):  # 'm'キーで移動モードに切り替え
                mode = 'move'
                print("移動モード")
                update_image()
            elif key == ord('t'):  # 't'キーで変形モードに切り替え
                mode = 'transform'
                print("変形モード")
                update_image()
            elif key == ord('a'):  # 'a'キーで図形補正モードに切り替え
                mode = 'adjust'
                print("図形補正モード")
                update_image()
            elif key == ord('c'):  # 'c'キーで現在のセグメントの座標を表示
                current_coordinates = get_current_segment_coordinates()
                print("現在のセグメントの座標:")
                for segment, coords in current_coordinates.items():
                    print(f"{segment}: {coords}")

        if selected_segment:
            print(f"選択されたセグメント: {selected_segment}")

    except KeyboardInterrupt:
        print("プログラムが停止されました。")

    cv2.destroyAllWindows()

移動モード
変形モード
図形補正モード
移動モード
図形補正モード
変形モード
変形モード
移動モード
変形モード
移動モード
変形モード
移動モード
図形補正モード
移動モード
変形モード


## rev26 移動、拡大の情報が残らないバグを修正+Undo機能追加

In [107]:
import cv2
import numpy as np

# 画像の読み込み
image_path = 'segment_test.png'
image = cv2.imread(image_path)

# 画像が正しく読み込まれたか確認
if image is None:
    print(f"Error: 画像が見つかりません。パスを確認してください: {image_path}")
else:
    # グローバル変数
    ref_point = []
    cropping = False
    moving = False
    transforming = False
    adjusting = False
    selected_segment = None
    mode = 'select'  # 初期モードは選択モード
    original_image = image.copy()  # 元の画像を保持
    bounding_box = None
    scale_factor = 1.0
    total_dx = 0
    total_dy = 0
    total_angle = 0
    history = []

    # 7セグメントディスプレイの各セグメントの座標を定義
    segments = {
        'A': np.array([[50, 40], [140, 40], [160, 60], [140, 80], [50, 80], [30, 60]], np.int32),
        'B': np.array([[160, 60], [180, 80], [180, 180], [160, 200], [140, 180], [140, 80]], np.int32),
        'C': np.array([[160, 200], [180, 220], [180, 320], [160, 340], [140, 320], [140, 220]], np.int32),
        'D': np.array([[50, 320], [140, 320], [160, 340], [140, 360], [50, 360], [30, 340]], np.int32),
        'E': np.array([[30, 200], [50, 220], [50, 320], [30, 340], [10, 320], [10, 220]], np.int32),
        'F': np.array([[30, 60], [50, 80], [50, 180], [30, 200], [10, 180], [10, 80]], np.int32),
        'G': np.array([[50, 180], [140, 180], [160, 200], [140, 220], [50, 220], [30, 200]], np.int32),
        'DP': np.array([[180, 360], [200, 360], [200, 380], [180, 380]], np.int32)  # 小数点のドット
    }

    # スケーリングと中央に寄せるための変換行列を定義
    def scale_and_center_segment(points, scale_factor, offset_x, offset_y):
        scaled_points = []
        for x, y in points:
            scaled_x = int(x * scale_factor + offset_x)
            scaled_y = int(y * scale_factor + offset_y)
            scaled_points.append([scaled_x, scaled_y])
        return np.array(scaled_points, np.int32)

    # スケーリングと中央に寄せたセグメントを保持する辞書
    scaled_segments = {segment: scale_and_center_segment(points, scale_factor, 0, 0) for segment, points in segments.items()}

    # 現在のセグメントの座標を取得する関数
    def get_current_segment_coordinates():
        return {segment: points.tolist() for segment, points in scaled_segments.items()}

    # セグメントGを基準に他のセグメントを再計算する関数
    def recalculate_segments(scale_factor, dx=0, dy=0, angle=0):
        g_points = segments['G']
        g_center = np.mean(g_points, axis=0)
        new_segments = {}
        for segment, points in segments.items():
            relative_points = points - g_center
            scaled_points = relative_points * scale_factor + g_center
            rotated_points = adjust_segment_tilt(scaled_points, angle)
            new_segments[segment] = scale_and_center_segment(rotated_points, 1, dx, dy)
        return new_segments

    # 7セグメントを囲う補助線を描画する関数
    def draw_bounding_box(image, segments):
        all_points = np.concatenate(list(segments.values()))
        x, y, w, h = cv2.boundingRect(all_points)
        cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 1, lineType=cv2.LINE_AA)
        return (x, y, x + w, y + h)

    # セグメントの傾きを調整する関数
    def adjust_segment_tilt(points, angle):
        center = np.mean(points, axis=0)
        rotation_matrix = cv2.getRotationMatrix2D(tuple(center), angle, 1.0)
        adjusted_points = cv2.transform(np.array([points]), rotation_matrix)[0]
        return adjusted_points

    # 画像を更新する関数
    def update_image():
        global image, scaled_segments, bounding_box
        image = original_image.copy()  # 元の画像に戻す
        for points in scaled_segments.values():
            cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
        bounding_box = draw_bounding_box(image, scaled_segments)
        cv2.imshow("image", image)

    # 操作履歴を保存する関数
    def save_history():
        global history, scaled_segments, scale_factor, total_dx, total_dy, total_angle
        history.append((scaled_segments.copy(), scale_factor, total_dx, total_dy, total_angle))

    # 操作履歴を元に戻す関数
    def undo():
        global history, scaled_segments, scale_factor, total_dx, total_dy, total_angle
        if history:
            scaled_segments, scale_factor, total_dx, total_dy, total_angle = history.pop()
            update_image()

    # マウスコールバック関数
    def shape_selection(event, x, y, flags, param):
        global ref_point, cropping, moving, transforming, adjusting, selected_segment, mode, image, scaled_segments, bounding_box, scale_factor, total_dx, total_dy, total_angle

        if mode == 'select':
            if event == cv2.EVENT_LBUTTONDOWN:
                ref_point = [(x, y)]
                cropping = True
                selected_segment = None

            elif event == cv2.EVENT_MOUSEMOVE:
                if cropping:
                    temp_image = image.copy()
                    for points in scaled_segments.values():
                        cv2.polylines(temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    bounding_box = draw_bounding_box(temp_image, scaled_segments)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if cropping:
                    ref_point.append((x, y))
                    cropping = False
                    for segment, points in scaled_segments.items():
                        if cv2.pointPolygonTest(points, ref_point[0], False) >= 0:
                            selected_segment = segment
                            break
                    update_image()
                    if selected_segment:
                        cv2.polylines(image, [scaled_segments[selected_segment]], isClosed=True, color=(0, 0, 255), thickness=2)
                    cv2.imshow("image", image)

        elif mode == 'move':
            if event == cv2.EVENT_LBUTTONDOWN:
                if bounding_box and bounding_box[0] <= x <= bounding_box[2] and bounding_box[1] <= y <= bounding_box[3]:
                    moving = True
                    ref_point = [(x, y)]
                    save_history()

            elif event == cv2.EVENT_MOUSEMOVE:
                if moving:
                    dx = x - ref_point[0][0]
                    dy = y - ref_point[0][1]
                    temp_image = original_image.copy()  # 元の画像に戻す
                    new_segments = {segment: points + [dx, dy] for segment, points in scaled_segments.items()}
                    for points in new_segments.values():
                        cv2.polylines(temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    bounding_box = draw_bounding_box(temp_image, new_segments)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if moving:
                    dx = x - ref_point[0][0]
                    dy = y - ref_point[0][1]
                    total_dx += dx
                    total_dy += dy
                    scaled_segments = {segment: points + [dx, dy] for segment, points in scaled_segments.items()}
                    moving = False
                    update_image()

        elif mode == 'transform':
            if event == cv2.EVENT_LBUTTONDOWN:
                if bounding_box and bounding_box[0] <= x <= bounding_box[2] and bounding_box[1] <= y <= bounding_box[3]:
                    transforming = True
                    ref_point = [(x, y)]
                    save_history()

            elif event == cv2.EVENT_MOUSEMOVE:
                if transforming:
                    dx = x - ref_point[0][0]
                    new_scale_factor = scale_factor * (1 + dx / 100.0)
                    temp_image = original_image.copy()  # 元の画像に戻す
                    new_segments = recalculate_segments(new_scale_factor, total_dx, total_dy, total_angle)
                    for points in new_segments.values():
                        cv2.polylines(temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    bounding_box = draw_bounding_box(temp_image, new_segments)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if transforming:
                    dx = x - ref_point[0][0]
                    scale_factor *= (1 + dx / 100.0)
                    scaled_segments = recalculate_segments(scale_factor, total_dx, total_dy, total_angle)
                    transforming = False
                    update_image()

        elif mode == 'adjust':
            if event == cv2.EVENT_LBUTTONDOWN:
                if bounding_box and bounding_box[0] <= x <= bounding_box[2] and bounding_box[1] <= y <= bounding_box[3]:
                    adjusting = True
                    ref_point = [(x, y)]
                    save_history()

            elif event == cv2.EVENT_MOUSEMOVE:
                if adjusting:
                    dx = x - ref_point[0][0]
                    angle = total_angle + dx / 5.0  # 角度を調整するためのスケール
                    temp_image = original_image.copy()  # 元の画像に戻す
                    new_segments = {segment: adjust_segment_tilt(points, angle) for segment, points in scaled_segments.items()}
                    for points in new_segments.values():
                        cv2.polylines(temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    bounding_box = draw_bounding_box(temp_image, new_segments)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if adjusting:
                    dx = x - ref_point[0][0]
                    total_angle += dx / 5.0  # 角度を調整するためのスケール
                    scaled_segments = {segment: adjust_segment_tilt(points, total_angle) for segment, points in scaled_segments.items()}
                    adjusting = False
                    update_image()

    # ウィンドウを作成し、マウスコールバックを設定
    cv2.namedWindow("image")
    cv2.setMouseCallback("image", shape_selection)

    try:
        # 画像を表示
        while True:
            cv2.imshow("image", image)
            key = cv2.waitKey(1) & 0xFF

            if key == 27:  # ESCキーで終了
                break
            elif key == ord('s'):  # 's'キーで選択モードに切り替え
                mode = 'select'
                print("選択モード")
                update_image()
            elif key == ord('m'):  # 'm'キーで移動モードに切り替え
                mode = 'move'
                print("移動モード")
                update_image()
            elif key == ord('t'):  # 't'キーで変形モードに切り替え
                mode = 'transform'
                print("変形モード")
                update_image()
            elif key == ord('a'):  # 'a'キーで図形補正モードに切り替え
                mode = 'adjust'
                print("図形補正モード")
                update_image()
            elif key == ord('c'):  # 'c'キーで現在のセグメントの座標を表示
                current_coordinates = get_current_segment_coordinates()
                print("現在のセグメントの座標:")
                for segment, coords in current_coordinates.items():
                    print(f"{segment}: {coords}")
            elif key == 26:  # Ctrl + Zで元に戻す
                undo()

        if selected_segment:
            print(f"選択されたセグメント: {selected_segment}")

    except KeyboardInterrupt:
        print("プログラムが停止されました。")

    cv2.destroyAllWindows()

移動モード
変形モード
移動モード
変形モード
移動モード
図形補正モード
移動モード
選択モード
選択されたセグメント: D


## rev30 ぬるぬる動く

In [2]:
import cv2
import numpy as np

# 画像の読み込み
image_path = 'C:/Users/Owner/Desktop/my_program/my_pattern_match/image/segment_test.png'
image = cv2.imread(image_path)

# 画像が正しく読み込まれたか確認
if image is None:
    print(f"Error: 画像が見つかりません。パスを確認してください: {image_path}")
else:
    # グローバル変数
    ref_point = []
    cropping = False
    moving = False
    transforming = False
    adjusting = False
    selected_segment = None
    mode = 'select'  # 初期モードは選択モード
    original_image = image.copy()  # 元の画像を保持
    temp_image = image.copy()  # 描画用の一時画像
    bounding_box = None
    scale_factor = 1.0
    total_dx = 0
    total_dy = 0
    total_angle = 0
    history = []

    # 7セグメントディスプレイの各セグメントの座標を定義
    segments = {
        'A': np.array([[50, 40], [140, 40], [160, 60], [140, 80], [50, 80], [30, 60]], np.int32),
        'B': np.array([[160, 60], [180, 80], [180, 180], [160, 200], [140, 180], [140, 80]], np.int32),
        'C': np.array([[160, 200], [180, 220], [180, 320], [160, 340], [140, 320], [140, 220]], np.int32),
        'D': np.array([[50, 320], [140, 320], [160, 340], [140, 360], [50, 360], [30, 340]], np.int32),
        'E': np.array([[30, 200], [50, 220], [50, 320], [30, 340], [10, 320], [10, 220]], np.int32),
        'F': np.array([[30, 60], [50, 80], [50, 180], [30, 200], [10, 180], [10, 80]], np.int32),
        'G': np.array([[50, 180], [140, 180], [160, 200], [140, 220], [50, 220], [30, 200]], np.int32),
        'DP': np.array([[180, 360], [200, 360], [200, 380], [180, 380]], np.int32)  # 小数点のドット
    }

    # スケーリングと中央に寄せるための変換行列を定義
    def scale_and_center_segment(points, scale_factor, offset_x, offset_y):
        scaled_points = []
        for x, y in points:
            scaled_x = int(x * scale_factor + offset_x)
            scaled_y = int(y * scale_factor + offset_y)
            scaled_points.append([scaled_x, scaled_y])
        return np.array(scaled_points, np.int32)

    # スケーリングと中央に寄せたセグメントを保持する辞書
    scaled_segments = {segment: scale_and_center_segment(points, scale_factor, 0, 0) for segment, points in segments.items()}

    # 現在のセグメントの座標を取得する関数
    def get_current_segment_coordinates():
        return {segment: points.tolist() for segment, points in scaled_segments.items()}

    # セグメントGを基準に他のセグメントを再計算する関数
    def recalculate_segments(scale_factor, dx=0, dy=0, angle=0):
        g_points = segments['G']
        g_center = np.mean(g_points, axis=0)
        new_segments = {}
        for segment, points in segments.items():
            relative_points = points - g_center
            scaled_points = relative_points * scale_factor + g_center
            rotated_points = adjust_segment_tilt(scaled_points, angle)
            new_segments[segment] = scale_and_center_segment(rotated_points, 1, dx, dy)
        return new_segments

    # 7セグメントを囲う補助線を描画する関数
    def draw_bounding_box(image, segments):
        all_points = np.concatenate(list(segments.values()))
        x, y, w, h = cv2.boundingRect(all_points)
        cv2.rectangle(image, (x, y), (x + w, y + h), (255, 255, 255), 1, lineType=cv2.LINE_AA)
        return (x, y, x + w, y + h)

    # セグメントの傾きを調整する関数
    def adjust_segment_tilt(points, angle):
        center = np.mean(points, axis=0)
        rotation_matrix = cv2.getRotationMatrix2D(tuple(center), angle, 1.0)
        adjusted_points = cv2.transform(np.array([points]), rotation_matrix)[0]
        return adjusted_points

    # 画像を更新する関数
    def update_image():
        global image, scaled_segments, bounding_box, temp_image
        temp_image = original_image.copy()  # 元の画像に戻す
        for points in scaled_segments.values():
            cv2.polylines(temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
        bounding_box = draw_bounding_box(temp_image, scaled_segments)
        cv2.imshow("image", temp_image)

    # 操作履歴を保存する関数
    def save_history():
        global history, scaled_segments, scale_factor, total_dx, total_dy, total_angle
        history.append((scaled_segments.copy(), scale_factor, total_dx, total_dy, total_angle))

    # 操作履歴を元に戻す関数
    def undo():
        global history, scaled_segments, scale_factor, total_dx, total_dy, total_angle
        if history:
            scaled_segments, scale_factor, total_dx, total_dy, total_angle = history.pop()
            update_image()

    # マウスコールバック関数
    def shape_selection(event, x, y, flags, param):
        global ref_point, cropping, moving, transforming, adjusting, selected_segment, mode, image, scaled_segments, bounding_box, scale_factor, total_dx, total_dy, total_angle, temp_image

        if mode == 'select':
            if event == cv2.EVENT_LBUTTONDOWN:
                ref_point = [(x, y)]
                cropping = True
                selected_segment = None

            elif event == cv2.EVENT_MOUSEMOVE:
                if cropping:
                    temp_image = original_image.copy()
                    for points in scaled_segments.values():
                        cv2.polylines(temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    bounding_box = draw_bounding_box(temp_image, scaled_segments)
                    cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if cropping:
                    ref_point.append((x, y))
                    cropping = False
                    for segment, points in scaled_segments.items():
                        if cv2.pointPolygonTest(points, ref_point[0], False) >= 0:
                            selected_segment = segment
                            break
                    update_image()
                    if selected_segment:
                        cv2.polylines(temp_image, [scaled_segments[selected_segment]], isClosed=True, color=(0, 0, 255), thickness=2)
                    cv2.imshow("image", temp_image)

        elif mode == 'move':
            if event == cv2.EVENT_LBUTTONDOWN:
                if bounding_box and bounding_box[0] <= x <= bounding_box[2] and bounding_box[1] <= y <= bounding_box[3]:
                    moving = True
                    ref_point = [(x, y)]
                    save_history()

            elif event == cv2.EVENT_MOUSEMOVE and moving:
                dx = x - ref_point[0][0]
                dy = y - ref_point[0][1]
                temp_image = original_image.copy()  # 元の画像に戻す
                new_segments = {segment: points + [dx, dy] for segment, points in scaled_segments.items()}
                for points in new_segments.values():
                    cv2.polylines(temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                bounding_box = draw_bounding_box(temp_image, new_segments)
                cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP and moving:
                dx = x - ref_point[0][0]
                dy = y - ref_point[0][1]
                total_dx += dx
                total_dy += dy
                scaled_segments = {segment: points + [dx, dy] for segment, points in scaled_segments.items()}
                moving = False
                update_image()

        elif mode == 'transform':
            if event == cv2.EVENT_LBUTTONDOWN:
                if bounding_box and bounding_box[0] <= x <= bounding_box[2] and bounding_box[1] <= y <= bounding_box[3]:
                    transforming = True
                    ref_point = [(x, y)]
                    save_history()

            elif event == cv2.EVENT_MOUSEMOVE and transforming:
                dx = x - ref_point[0][0]
                new_scale_factor = scale_factor * (1 + dx / 100.0)
                temp_image = original_image.copy()  # 元の画像に戻す
                new_segments = recalculate_segments(new_scale_factor, total_dx, total_dy, total_angle)
                for points in new_segments.values():
                    cv2.polylines(temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                bounding_box = draw_bounding_box(temp_image, new_segments)
                cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP and transforming:
                dx = x - ref_point[0][0]
                scale_factor *= (1 + dx / 100.0)
                scaled_segments = recalculate_segments(scale_factor, total_dx, total_dy, total_angle)
                transforming = False
                update_image()

        elif mode == 'adjust':
            if event == cv2.EVENT_LBUTTONDOWN:
                if bounding_box and bounding_box[0] <= x <= bounding_box[2] and bounding_box[1] <= y <= bounding_box[3]:
                    adjusting = True
                    ref_point = [(x, y)]
                    save_history()

            elif event == cv2.EVENT_MOUSEMOVE and adjusting:
                dx = x - ref_point[0][0]
                angle = total_angle + dx / 5.0  # 角度を調整するためのスケール
                temp_image = original_image.copy()  # 元の画像に戻す
                new_segments = {segment: adjust_segment_tilt(points, angle) for segment, points in scaled_segments.items()}
                for points in new_segments.values():
                    cv2.polylines(temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                bounding_box = draw_bounding_box(temp_image, new_segments)
                cv2.imshow("image", temp_image)

            elif event == cv2.EVENT_LBUTTONUP and adjusting:
                dx = x - ref_point[0][0]
                total_angle += dx / 5.0  # 角度を調整するためのスケール
                scaled_segments = {segment: adjust_segment_tilt(points, total_angle) for segment, points in scaled_segments.items()}
                adjusting = False
                update_image()

    # ウィンドウを作成し、マウスコールバックを設定
    cv2.namedWindow("image")
    cv2.setMouseCallback("image", shape_selection)

    try:
        # 画像を表示
        while True:
            cv2.imshow("image", temp_image)
            key = cv2.waitKey(1) & 0xFF

            if key == 27:  # ESCキーで終了
                break
            elif key == ord('s'):  # 's'キーで選択モードに切り替え
                mode = 'select'
                print("選択モード")
                update_image()
            elif key == ord('m'):  # 'm'キーで移動モードに切り替え
                mode = 'move'
                print("移動モード")
                update_image()
            elif key == ord('t'):  # 't'キーで変形モードに切り替え
                mode = 'transform'
                print("変形モード")
                update_image()
            elif key == ord('a'):  # 'a'キーで図形補正モードに切り替え
                mode = 'adjust'
                print("図形補正モード")
                update_image()
            elif key == ord('c'):  # 'c'キーで現在のセグメントの座標を表示
                current_coordinates = get_current_segment_coordinates()
                print("現在のセグメントの座標:")
                for segment, coords in current_coordinates.items():
                    print(f"{segment}: {coords}")
            elif key == 26:  # Ctrl + Zで元に戻す
                undo()

        if selected_segment:
            print(f"選択されたセグメント: {selected_segment}")

    except KeyboardInterrupt:
        print("プログラムが停止されました。")

    cv2.destroyAllWindows()

移動モード
変形モード
移動モード
変形モード
移動モード
選択されたセグメント: A


## rev32 class メソッド化

In [4]:
import cv2
import numpy as np

class SegmentDisplay:
    def __init__(self, image, segments, scale_factor=1.0):
        self.original_image = image
        self.temp_image = image.copy()
        self.segments = segments
        self.scale_factor = scale_factor
        self.scaled_segments = self.recalculate_segments()
        self.current_coordinates = None
        self.ref_point = []
        self.cropping = False
        self.moving = False
        self.transforming = False
        self.adjusting = False
        self.selected_segment = None
        self.mode = 'select'
        self.bounding_box = None
        self.total_dx = 0
        self.total_dy = 0
        self.total_angle = 0
        self.history = []

    def scale_and_center_segment(self, points, scale_factor, offset_x, offset_y):
        scaled_points = []
        for x, y in points:
            scaled_x = int(x * scale_factor + offset_x)
            scaled_y = int(y * scale_factor + offset_y)
            scaled_points.append([scaled_x, scaled_y])
        return np.array(scaled_points, np.int32)

    def adjust_segment_tilt(self, points, angle):
        center = np.mean(points, axis=0)
        rotation_matrix = cv2.getRotationMatrix2D(tuple(center), angle, 1.0)
        adjusted_points = cv2.transform(np.array([points]), rotation_matrix)[0]
        return adjusted_points

    def recalculate_segments(self, scale_factor=None, dx=0, dy=0, angle=0):
        if scale_factor is None:
            scale_factor = self.scale_factor
        g_points = self.segments['G']
        g_center = np.mean(g_points, axis=0)
        new_segments = {}
        for segment, points in self.segments.items():
            relative_points = points - g_center
            scaled_points = relative_points * scale_factor + g_center
            rotated_points = self.adjust_segment_tilt(scaled_points, angle)
            new_segments[segment] = self.scale_and_center_segment(rotated_points, 1, dx, dy)
        return new_segments

    def draw_bounding_box(self, image, segments):
        all_points = np.concatenate(list(segments.values()))
        x, y, w, h = cv2.boundingRect(all_points)
        cv2.rectangle(image, (x, y), (x + w, y + h), (255, 255, 255), 1, lineType=cv2.LINE_AA)
        return (x, y, x + w, y + h)

    def update_image(self):
        self.temp_image = self.original_image.copy()
        for points in self.scaled_segments.values():
            cv2.polylines(self.temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
        self.bounding_box = self.draw_bounding_box(self.temp_image, self.scaled_segments)
        cv2.imshow("image", self.temp_image)

    def save_history(self):
        self.history.append((self.scaled_segments.copy(), self.scale_factor, self.total_dx, self.total_dy, self.total_angle))

    def undo(self):
        if self.history:
            self.scaled_segments, self.scale_factor, self.total_dx, self.total_dy, self.total_angle = self.history.pop()
            self.update_image()

    def shape_selection(self, event, x, y, flags, param):
        if self.mode == 'select':
            if event == cv2.EVENT_LBUTTONDOWN:
                self.ref_point = [(x, y)]
                self.cropping = True
                self.selected_segment = None

            elif event == cv2.EVENT_MOUSEMOVE:
                if self.cropping:
                    self.temp_image = self.original_image.copy()
                    for points in self.scaled_segments.values():
                        cv2.polylines(self.temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                    self.bounding_box = self.draw_bounding_box(self.temp_image, self.scaled_segments)
                    cv2.imshow("image", self.temp_image)

            elif event == cv2.EVENT_LBUTTONUP:
                if self.cropping:
                    self.ref_point.append((x, y))
                    self.cropping = False
                    for segment, points in self.scaled_segments.items():
                        if cv2.pointPolygonTest(points, self.ref_point[0], False) >= 0:
                            self.selected_segment = segment
                            break
                    self.update_image()
                    if self.selected_segment:
                        cv2.polylines(self.temp_image, [self.scaled_segments[self.selected_segment]], isClosed=True, color=(0, 0, 255), thickness=2)
                    cv2.imshow("image", self.temp_image)

        elif self.mode == 'move':
            if event == cv2.EVENT_LBUTTONDOWN:
                if self.bounding_box and self.bounding_box[0] <= x <= self.bounding_box[2] and self.bounding_box[1] <= y <= self.bounding_box[3]:
                    self.moving = True
                    self.ref_point = [(x, y)]
                    self.save_history()

            elif event == cv2.EVENT_MOUSEMOVE and self.moving:
                dx = x - self.ref_point[0][0]
                dy = y - self.ref_point[0][1]
                self.temp_image = self.original_image.copy()
                new_segments = {segment: points + [dx, dy] for segment, points in self.scaled_segments.items()}
                for points in new_segments.values():
                    cv2.polylines(self.temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                self.bounding_box = self.draw_bounding_box(self.temp_image, new_segments)
                cv2.imshow("image", self.temp_image)

            elif event == cv2.EVENT_LBUTTONUP and self.moving:
                dx = x - self.ref_point[0][0]
                dy = y - self.ref_point[0][1]
                self.total_dx += dx
                self.total_dy += dy
                self.scaled_segments = {segment: points + [dx, dy] for segment, points in self.scaled_segments.items()}
                self.moving = False
                self.update_image()

        elif self.mode == 'transform':
            if event == cv2.EVENT_LBUTTONDOWN:
                if self.bounding_box and self.bounding_box[0] <= x <= self.bounding_box[2] and self.bounding_box[1] <= y <= self.bounding_box[3]:
                    self.transforming = True
                    self.ref_point = [(x, y)]
                    self.save_history()

            elif event == cv2.EVENT_MOUSEMOVE and self.transforming:
                dx = x - self.ref_point[0][0]
                new_scale_factor = self.scale_factor * (1 + dx / 100.0)
                self.temp_image = self.original_image.copy()
                new_segments = self.recalculate_segments(new_scale_factor, self.total_dx, self.total_dy, self.total_angle)
                for points in new_segments.values():
                    cv2.polylines(self.temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                self.bounding_box = self.draw_bounding_box(self.temp_image, new_segments)
                cv2.imshow("image", self.temp_image)

            elif event == cv2.EVENT_LBUTTONUP and self.transforming:
                dx = x - self.ref_point[0][0]
                self.scale_factor *= (1 + dx / 100.0)
                self.scaled_segments = self.recalculate_segments(self.scale_factor, self.total_dx, self.total_dy, self.total_angle)
                self.transforming = False
                self.update_image()

        elif self.mode == 'adjust':
            if event == cv2.EVENT_LBUTTONDOWN:
                if self.bounding_box and self.bounding_box[0] <= x <= self.bounding_box[2] and self.bounding_box[1] <= y <= self.bounding_box[3]:
                    self.adjusting = True
                    self.ref_point = [(x, y)]
                    self.save_history()

            elif event == cv2.EVENT_MOUSEMOVE and self.adjusting:
                dx = x - self.ref_point[0][0]
                angle = self.total_angle + dx / 5.0
                self.temp_image = self.original_image.copy()
                new_segments = {segment: self.adjust_segment_tilt(points, angle) for segment, points in self.scaled_segments.items()}
                for points in new_segments.values():
                    cv2.polylines(self.temp_image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                self.bounding_box = self.draw_bounding_box(self.temp_image, new_segments)
                cv2.imshow("image", self.temp_image)

            elif event == cv2.EVENT_LBUTTONUP and self.adjusting:
                dx = x - self.ref_point[0][0]
                self.total_angle += dx / 5.0
                self.scaled_segments = {segment: self.adjust_segment_tilt(points, self.total_angle) for segment, points in self.scaled_segments.items()}
                self.adjusting = False
                self.update_image()

    def run(self):
        cv2.namedWindow("image")
        cv2.setMouseCallback("image", self.shape_selection)

        try:
            while True:
                cv2.imshow("image", self.temp_image)
                key = cv2.waitKey(1) & 0xFF

                if key == 27:  # ESCキーで終了
                    break
                elif key == ord('s'):  # 's'キーで選択モードに切り替え
                    self.mode = 'select'
                    print("選択モード")
                    self.update_image()
                elif key == ord('m'):  # 'm'キーで移動モードに切り替え
                    self.mode = 'move'
                    print("移動モード")
                    self.update_image()
                elif key == ord('t'):  # 't'キーで変形モードに切り替え
                    self.mode = 'transform'
                    print("変形モード")
                    self.update_image()
                elif key == ord('a'):  # 'a'キーで図形補正モードに切り替え
                    self.mode = 'adjust'
                    print("図形補正モード")
                    self.update_image()
                elif key == ord('c'):  # 'c'キーで現在のセグメントの座標を表示
                    current_coordinates = {segment: points.tolist() for segment, points in self.scaled_segments.items()}
                    self.current_coordinates = current_coordinates
                    print("現在のセグメントの座標:")
                    for segment, coords in current_coordinates.items():
                        print(f"{segment}: {coords}")
                elif key == 26:  # Ctrl + Zで元に戻す
                    self.undo()

            if self.selected_segment:
                print(f"選択されたセグメント: {self.selected_segment}")

        except KeyboardInterrupt:
            print("プログラムが停止されました。")

        cv2.destroyAllWindows()

    def get_current_segment(self):
        return self.current_coordinates

メインの実行

In [5]:

# 画像の読み込み
image_path = 'C:/Users/Owner/Desktop/my_program/my_pattern_match/image/segment_test.png'
image = cv2.imread(image_path)

# 画像が正しく読み込まれたか確認
if image is None:
    print(f"Error: 画像が見つかりません。パスを確認してください: {image_path}")
else:
    # 7セグメントディスプレイの各セグメントの座標を定義
    segments = {
        'A': np.array([[50, 40], [140, 40], [160, 60], [140, 80], [50, 80], [30, 60]], np.int32),
        'B': np.array([[160, 60], [180, 80], [180, 180], [160, 200], [140, 180], [140, 80]], np.int32),
        'C': np.array([[160, 200], [180, 220], [180, 320], [160, 340], [140, 320], [140, 220]], np.int32),
        'D': np.array([[50, 320], [140, 320], [160, 340], [140, 360], [50, 360], [30, 340]], np.int32),
        'E': np.array([[30, 200], [50, 220], [50, 320], [30, 340], [10, 320], [10, 220]], np.int32),
        'F': np.array([[30, 60], [50, 80], [50, 180], [30, 200], [10, 180], [10, 80]], np.int32),
        'G': np.array([[50, 180], [140, 180], [160, 200], [140, 220], [50, 220], [30, 200]], np.int32),
        'DP': np.array([[180, 360], [200, 360], [200, 380], [180, 380]], np.int32)  # 小数点のドット
    }

    # SegmentDisplayオブジェクトを作成し、実行
    display = SegmentDisplay(image, segments)
    display.run()

変形モード
移動モード
現在のセグメントの座標:
A: [[116, 59], [152, 59], [160, 67], [152, 75], [116, 75], [108, 67]]
B: [[160, 67], [168, 75], [168, 115], [160, 123], [152, 115], [152, 75]]
C: [[160, 123], [168, 131], [168, 171], [160, 179], [152, 171], [152, 131]]
D: [[116, 171], [152, 171], [160, 179], [152, 187], [116, 187], [108, 179]]
E: [[108, 123], [116, 131], [116, 171], [108, 179], [100, 171], [100, 131]]
F: [[108, 67], [116, 75], [116, 115], [108, 123], [100, 115], [100, 75]]
G: [[116, 115], [152, 115], [160, 123], [152, 131], [116, 131], [108, 123]]
DP: [[168, 187], [176, 187], [176, 195], [168, 195]]
現在のセグメントの座標:
A: [[116, 59], [152, 59], [160, 67], [152, 75], [116, 75], [108, 67]]
B: [[160, 67], [168, 75], [168, 115], [160, 123], [152, 115], [152, 75]]
C: [[160, 123], [168, 131], [168, 171], [160, 179], [152, 171], [152, 131]]
D: [[116, 171], [152, 171], [160, 179], [152, 187], [116, 187], [108, 179]]
E: [[108, 123], [116, 131], [116, 171], [108, 179], [100, 171], [100, 131]]
F: [[108, 67], [11

In [6]:
display.get_current_segment()

{'A': [[116, 59], [152, 59], [160, 67], [152, 75], [116, 75], [108, 67]],
 'B': [[160, 67], [168, 75], [168, 115], [160, 123], [152, 115], [152, 75]],
 'C': [[160, 123], [168, 131], [168, 171], [160, 179], [152, 171], [152, 131]],
 'D': [[116, 171], [152, 171], [160, 179], [152, 187], [116, 187], [108, 179]],
 'E': [[108, 123], [116, 131], [116, 171], [108, 179], [100, 171], [100, 131]],
 'F': [[108, 67], [116, 75], [116, 115], [108, 123], [100, 115], [100, 75]],
 'G': [[116, 115], [152, 115], [160, 123], [152, 131], [116, 131], [108, 123]],
 'DP': [[168, 187], [176, 187], [176, 195], [168, 195]]}

In [7]:
## rev33 Segmentクラス　SegmentDisplayクラス実装

In [8]:
import cv2
import numpy as np

class Segment:
    def __init__(self, segments, scale_factor=1.0, dx=0, dy=0, angle=0):
        self.original_segments = segments
        self.scale_factor = scale_factor
        self.dx = dx
        self.dy = dy
        self.angle = angle
        self.scaled_segments = self.recalculate_segments()

    def scale_and_center_segment(self, points, scale_factor, offset_x, offset_y):
        scaled_points = []
        for x, y in points:
            scaled_x = int(x * scale_factor + offset_x)
            scaled_y = int(y * scale_factor + offset_y)
            scaled_points.append([scaled_x, scaled_y])
        return np.array(scaled_points, np.int32)

    def adjust_segment_tilt(self, points, angle):
        center = np.mean(points, axis=0)
        rotation_matrix = cv2.getRotationMatrix2D(tuple(center), angle, 1.0)
        adjusted_points = cv2.transform(np.array([points]), rotation_matrix)[0]
        return adjusted_points

    def recalculate_segments(self):
        g_points = self.original_segments['G']
        g_center = np.mean(g_points, axis=0)
        new_segments = {}
        for segment, points in self.original_segments.items():
            relative_points = points - g_center
            scaled_points = relative_points * self.scale_factor + g_center
            rotated_points = self.adjust_segment_tilt(scaled_points, self.angle)
            new_segments[segment] = self.scale_and_center_segment(rotated_points, 1, self.dx, self.dy)
        return new_segments

    def draw(self, image):
        for points in self.scaled_segments.values():
            cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)

    def move(self, dx, dy):
        self.dx += dx
        self.dy += dy
        self.scaled_segments = self.recalculate_segments()

    def transform(self, scale_factor):
        self.scale_factor *= scale_factor
        self.scaled_segments = self.recalculate_segments()

    def adjust(self, angle):
        self.angle += angle
        self.scaled_segments = self.recalculate_segments()

class SegmentDisplay:
    def __init__(self, image):
        self.original_image = image
        self.temp_image = image.copy()
        self.segments = []

    def add_segment(self, segment):
        self.segments.append(segment)

    def update_image(self):
        self.temp_image = self.original_image.copy()
        for segment in self.segments:
            segment.draw(self.temp_image)
        cv2.imshow("image", self.temp_image)

    def run(self):
        cv2.namedWindow("image")
        self.update_image()

        try:
            while True:
                key = cv2.waitKey(1) & 0xFF
                if key == 27:  # ESCキーで終了
                    break
                elif key == ord('s'):  # 's'キーで選択モードに切り替え
                    self.mode = 'select'
                    print("選択モード")
                    self.update_image()
                elif key == ord('m'):  # 'm'キーで移動モードに切り替え
                    self.mode = 'move'
                    print("移動モード")
                    self.update_image()
                elif key == ord('t'):  # 't'キーで変形モードに切り替え
                    self.mode = 'transform'
                    print("変形モード")
                    self.update_image()
                elif key == ord('a'):  # 'a'キーで図形補正モードに切り替え
                    self.mode = 'adjust'
                    print("図形補正モード")
                    self.update_image()
                elif key == ord('c'):  # 'c'キーで現在のセグメントの座標を表示
                    current_coordinates = {segment: points.tolist() for segment, points in self.scaled_segments.items()}
                    self.current_coordinates = current_coordinates
                    print("現在のセグメントの座標:")
                    for segment, coords in current_coordinates.items():
                        print(f"{segment}: {coords}")
                elif key == 26:  # Ctrl + Zで元に戻す
                    self.undo()
                # 他のキー操作を追加可能

        except KeyboardInterrupt:
            print("プログラムが停止されました。")

        cv2.destroyAllWindows()




In [10]:
# 画像の読み込み
image_path = 'C:/Users/Owner/Desktop/my_program/my_pattern_match/image/segment_test.png'
image = cv2.imread(image_path)

# 画像が正しく読み込まれたか確認
if image is None:
    print(f"Error: 画像が見つかりません。パスを確認してください: {image_path}")
    exit()

# 7セグメントディスプレイの各セグメントの座標を定義
segments = {
    'A': np.array([[50, 40], [140, 40], [160, 60], [140, 80], [50, 80], [30, 60]], np.int32),
    'B': np.array([[160, 60], [180, 80], [180, 180], [160, 200], [140, 180], [140, 80]], np.int32),
    'C': np.array([[160, 200], [180, 220], [180, 320], [160, 340], [140, 320], [140, 220]], np.int32),
    'D': np.array([[50, 320], [140, 320], [160, 340], [140, 360], [50, 360], [30, 340]], np.int32),
    'E': np.array([[30, 200], [50, 220], [50, 320], [30, 340], [10, 320], [10, 220]], np.int32),
    'F': np.array([[30, 60], [50, 80], [50, 180], [30, 200], [10, 180], [10, 80]], np.int32),
    'G': np.array([[50, 180], [140, 180], [160, 200], [140, 220], [50, 220], [30, 200]], np.int32),
    'DP': np.array([[180, 360], [200, 360], [200, 380], [180, 380]], np.int32)  # 小数点のドット
}


# SegmentDisplayオブジェクトを作成
display = SegmentDisplay(image)

# Segmentオブジェクトを作成し、追加
segment = Segment(segments)
display.add_segment(segment)

# 実行
display.run()

AttributeError: 'SegmentDisplay' object has no attribute 'scaled_segments'

## rev34 セグメントアクセスする変数修正

In [11]:
import cv2
import numpy as np

class Segment:
    def __init__(self, segments, scale_factor=1.0, dx=0, dy=0, angle=0):
        self.original_segments = segments
        self.scale_factor = scale_factor
        self.dx = dx
        self.dy = dy
        self.angle = angle
        self.scaled_segments = self.recalculate_segments()

    def scale_and_center_segment(self, points, scale_factor, offset_x, offset_y):
        scaled_points = []
        for x, y in points:
            scaled_x = int(x * scale_factor + offset_x)
            scaled_y = int(y * scale_factor + offset_y)
            scaled_points.append([scaled_x, scaled_y])
        return np.array(scaled_points, np.int32)

    def adjust_segment_tilt(self, points, angle):
        center = np.mean(points, axis=0)
        rotation_matrix = cv2.getRotationMatrix2D(tuple(center), angle, 1.0)
        adjusted_points = cv2.transform(np.array([points]), rotation_matrix)[0]
        return adjusted_points

    def recalculate_segments(self):
        g_points = self.original_segments['G']
        g_center = np.mean(g_points, axis=0)
        new_segments = {}
        for segment, points in self.original_segments.items():
            relative_points = points - g_center
            scaled_points = relative_points * self.scale_factor + g_center
            rotated_points = self.adjust_segment_tilt(scaled_points, self.angle)
            new_segments[segment] = self.scale_and_center_segment(rotated_points, 1, self.dx, self.dy)
        return new_segments

    def draw(self, image):
        for points in self.scaled_segments.values():
            cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)

    def move(self, dx, dy):
        self.dx += dx
        self.dy += dy
        self.scaled_segments = self.recalculate_segments()

    def transform(self, scale_factor):
        self.scale_factor *= scale_factor
        self.scaled_segments = self.recalculate_segments()

    def adjust(self, angle):
        self.angle += angle
        self.scaled_segments = self.recalculate_segments()

    def get_coordinates(self):
        return {segment: points.tolist() for segment, points in self.scaled_segments.items()}

class SegmentDisplay:
    def __init__(self, image):
        self.original_image = image
        self.temp_image = image.copy()
        self.segments = []
        self.mode = 'select'

    def add_segment(self, segment):
        self.segments.append(segment)

    def update_image(self):
        self.temp_image = self.original_image.copy()
        for segment in self.segments:
            segment.draw(self.temp_image)
        cv2.imshow("image", self.temp_image)

    def undo(self):
        # 実装例: セグメントの状態を元に戻す処理
        pass

    def run(self):
        cv2.namedWindow("image")
        self.update_image()

        try:
            while True:
                key = cv2.waitKey(1) & 0xFF
                if key == 27:  # ESCキーで終了
                    break
                elif key == ord('s'):  # 's'キーで選択モードに切り替え
                    self.mode = 'select'
                    print("選択モード")
                    self.update_image()
                elif key == ord('m'):  # 'm'キーで移動モードに切り替え
                    self.mode = 'move'
                    print("移動モード")
                    self.update_image()
                elif key == ord('t'):  # 't'キーで変形モードに切り替え
                    self.mode = 'transform'
                    print("変形モード")
                    self.update_image()
                elif key == ord('a'):  # 'a'キーで図形補正モードに切り替え
                    self.mode = 'adjust'
                    print("図形補正モード")
                    self.update_image()
                elif key == ord('c'):  # 'c'キーで現在のセグメントの座標を表示
                    for segment in self.segments:
                        current_coordinates = segment.get_coordinates()
                        print("現在のセグメントの座標:")
                        for seg, coords in current_coordinates.items():
                            print(f"{seg}: {coords}")
                elif key == 26:  # Ctrl + Zで元に戻す
                    self.undo()
                # 他のキー操作を追加可能

        except KeyboardInterrupt:
            print("プログラムが停止されました。")

        cv2.destroyAllWindows()


In [12]:

# 画像の読み込み
image_path = 'C:/Users/Owner/Desktop/my_program/my_pattern_match/image/segment_test.png'
image = cv2.imread(image_path)

# 画像が正しく読み込まれたか確認
if image is None:
    print(f"Error: 画像が見つかりません。パスを確認してください: {image_path}")
else:
    # 7セグメントディスプレイの各セグメントの座標を定義
    segments = {
        'A': np.array([[50, 40], [140, 40], [160, 60], [140, 80], [50, 80], [30, 60]], np.int32),
        'B': np.array([[160, 60], [180, 80], [180, 180], [160, 200], [140, 180], [140, 80]], np.int32),
        'C': np.array([[160, 200], [180, 220], [180, 320], [160, 340], [140, 320], [140, 220]], np.int32),
        'D': np.array([[50, 320], [140, 320], [160, 340], [140, 360], [50, 360], [30, 340]], np.int32),
        'E': np.array([[30, 200], [50, 220], [50, 320], [30, 340], [10, 320], [10, 220]], np.int32),
        'F': np.array([[30, 60], [50, 80], [50, 180], [30, 200], [10, 180], [10, 80]], np.int32),
        'G': np.array([[50, 180], [140, 180], [160, 200], [140, 220], [50, 220], [30, 200]], np.int32),
        'DP': np.array([[180, 360], [200, 360], [200, 380], [180, 380]], np.int32)  # 小数点のドット
    }

    # SegmentDisplayオブジェクトを作成
    display = SegmentDisplay(image)

    # Segmentオブジェクトを作成し、追加
    segment = Segment(segments)
    display.add_segment(segment)

    # 実行
    display.run()

変形モード
移動モード
現在のセグメントの座標:
A: [[50, 40], [140, 40], [160, 60], [140, 80], [50, 80], [30, 60]]
B: [[160, 60], [180, 80], [180, 180], [160, 200], [140, 180], [140, 80]]
C: [[160, 200], [180, 220], [180, 320], [160, 340], [140, 320], [140, 220]]
D: [[50, 320], [140, 320], [160, 340], [140, 360], [50, 360], [30, 340]]
E: [[30, 200], [50, 220], [50, 320], [30, 340], [10, 320], [10, 220]]
F: [[30, 60], [50, 80], [50, 180], [30, 200], [10, 180], [10, 80]]
G: [[50, 180], [140, 180], [160, 200], [140, 220], [50, 220], [30, 200]]
DP: [[180, 360], [200, 360], [200, 380], [180, 380]]


## rev35 イベントをSegmentクラスに反映

In [14]:
import cv2
import numpy as np

class Segment:
    def __init__(self, segments, scale_factor=1.0, dx=0, dy=0, angle=0):
        self.original_segments = segments
        self.scale_factor = scale_factor
        self.dx = dx
        self.dy = dy
        self.angle = angle
        self.scaled_segments = self.recalculate_segments()
        self.selected = False

    def scale_and_center_segment(self, points, scale_factor, offset_x, offset_y):
        scaled_points = []
        for x, y in points:
            scaled_x = int(x * scale_factor + offset_x)
            scaled_y = int(y * scale_factor + offset_y)
            scaled_points.append([scaled_x, scaled_y])
        return np.array(scaled_points, np.int32)

    def adjust_segment_tilt(self, points, angle):
        center = np.mean(points, axis=0)
        rotation_matrix = cv2.getRotationMatrix2D(tuple(center), angle, 1.0)
        adjusted_points = cv2.transform(np.array([points]), rotation_matrix)[0]
        return adjusted_points

    def recalculate_segments(self):
        g_points = self.original_segments['G']
        g_center = np.mean(g_points, axis=0)
        new_segments = {}
        for segment, points in self.original_segments.items():
            relative_points = points - g_center
            scaled_points = relative_points * self.scale_factor + g_center
            rotated_points = self.adjust_segment_tilt(scaled_points, self.angle)
            new_segments[segment] = self.scale_and_center_segment(rotated_points, 1, self.dx, self.dy)
        return new_segments

    def draw(self, image):
        color = (0, 255, 0) if not self.selected else (0, 0, 255)
        for points in self.scaled_segments.values():
            cv2.polylines(image, [points], isClosed=True, color=color, thickness=2)

    def move(self, dx, dy):
        self.dx += dx
        self.dy += dy
        self.scaled_segments = self.recalculate_segments()

    def transform(self, scale_factor):
        self.scale_factor *= scale_factor
        self.scaled_segments = self.recalculate_segments()

    def adjust(self, angle):
        self.angle += angle
        self.scaled_segments = self.recalculate_segments()

    def get_coordinates(self):
        return {segment: points.tolist() for segment, points in self.scaled_segments.items()}

    def handle_mouse_event(self, event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            for points in self.scaled_segments.values():
                if cv2.pointPolygonTest(points, (x, y), False) >= 0:
                    self.selected = True
                    self.ref_point = (x, y)
                    break
        elif event == cv2.EVENT_MOUSEMOVE and self.selected:
            dx = x - self.ref_point[0]
            dy = y - self.ref_point[1]
            self.move(dx, dy)
            self.ref_point = (x, y)
        elif event == cv2.EVENT_LBUTTONUP:
            self.selected = False

class SegmentDisplay:
    def __init__(self, image):
        self.original_image = image
        self.temp_image = image.copy()
        self.segments = []
        self.mode = 'select'

    def add_segment(self, segment):
        self.segments.append(segment)

    def update_image(self):
        self.temp_image = self.original_image.copy()
        for segment in self.segments:
            segment.draw(self.temp_image)
        cv2.imshow("image", self.temp_image)

    def run(self):
        cv2.namedWindow("image")
        cv2.setMouseCallback("image", self.handle_mouse_event)
        self.update_image()

        try:
            while True:
                key = cv2.waitKey(1) & 0xFF
                if key == 27:  # ESCキーで終了
                    break
                elif key == ord('s'):  # 's'キーで選択モードに切り替え
                    self.mode = 'select'
                    print("選択モード")
                    self.update_image()
                elif key == ord('m'):  # 'm'キーで移動モードに切り替え
                    self.mode = 'move'
                    print("移動モード")
                    self.update_image()
                elif key == ord('t'):  # 't'キーで変形モードに切り替え
                    self.mode = 'transform'
                    print("変形モード")
                    self.update_image()
                elif key == ord('a'):  # 'a'キーで図形補正モードに切り替え
                    self.mode = 'adjust'
                    print("図形補正モード")
                    self.update_image()
                elif key == ord('c'):  # 'c'キーで現在のセグメントの座標を表示
                    for segment in self.segments:
                        current_coordinates = segment.get_coordinates()
                        print("現在のセグメントの座標:")
                        for seg, coords in current_coordinates.items():
                            print(f"{seg}: {coords}")
                # 他のキー操作を追加可能

        except KeyboardInterrupt:
            print("プログラムが停止されました。")

        cv2.destroyAllWindows()

    def handle_mouse_event(self, event, x, y, flags, param):
        for segment in self.segments:
            segment.handle_mouse_event(event, x, y, flags, param)

# 画像の読み込み
image_path = 'C:/Users/Owner/Desktop/my_program/my_pattern_match/image/segment_test.png'
image = cv2.imread(image_path)

# 画像が正しく読み込まれたか確認
if image is None:
    print(f"Error: 画像が見つかりません。パスを確認してください: {image_path}")
else:
    # 7セグメントディスプレイの各セグメントの座標を定義
    segments = {
        'A': np.array([[50, 40], [140, 40], [160, 60], [140, 80], [50, 80], [30, 60]], np.int32),
        'B': np.array([[160, 60], [180, 80], [180, 180], [160, 200], [140, 180], [140, 80]], np.int32),
        'C': np.array([[160, 200], [180, 220], [180, 320], [160, 340], [140, 320], [140, 220]], np.int32),
        'D': np.array([[50, 320], [140, 320], [160, 340], [140, 360], [50, 360], [30, 340]], np.int32),
        'E': np.array([[30, 200], [50, 220], [50, 320], [30, 340], [10, 320], [10, 220]], np.int32),
        'F': np.array([[30, 60], [50, 80], [50, 180], [30, 200], [10, 180], [10, 80]], np.int32),
        'G': np.array([[50, 180], [140, 180], [160, 200], [140, 220], [50, 220], [30, 200]], np.int32),
        'DP': np.array([[180, 360], [200, 360], [200, 380], [180, 380]], np.int32)  # 小数点のドット
    }

    # SegmentDisplayオブジェクトを作成
    display = SegmentDisplay(image)

    # Segmentオブジェクトを作成し、追加
    segment = Segment(segments)
    display.add_segment(segment)

    # 実行
    display.run()

移動モード
変形モード
移動モード
変形モード
移動モード


## rev36 各モードのイベントに対応

In [15]:
import cv2
import numpy as np

class Segment:
    def __init__(self, segments, scale_factor=1.0, dx=0, dy=0, angle=0):
        self.original_segments = segments
        self.scale_factor = scale_factor
        self.dx = dx
        self.dy = dy
        self.angle = angle
        self.scaled_segments = self.recalculate_segments()
        self.selected = False
        self.ref_point = None

    def scale_and_center_segment(self, points, scale_factor, offset_x, offset_y):
        scaled_points = []
        for x, y in points:
            scaled_x = int(x * scale_factor + offset_x)
            scaled_y = int(y * scale_factor + offset_y)
            scaled_points.append([scaled_x, scaled_y])
        return np.array(scaled_points, np.int32)

    def adjust_segment_tilt(self, points, angle):
        center = np.mean(points, axis=0)
        rotation_matrix = cv2.getRotationMatrix2D(tuple(center), angle, 1.0)
        adjusted_points = cv2.transform(np.array([points]), rotation_matrix)[0]
        return adjusted_points

    def recalculate_segments(self):
        g_points = self.original_segments['G']
        g_center = np.mean(g_points, axis=0)
        new_segments = {}
        for segment, points in self.original_segments.items():
            relative_points = points - g_center
            scaled_points = relative_points * self.scale_factor + g_center
            rotated_points = self.adjust_segment_tilt(scaled_points, self.angle)
            new_segments[segment] = self.scale_and_center_segment(rotated_points, 1, self.dx, self.dy)
        return new_segments

    def draw(self, image):
        color = (0, 255, 0) if not self.selected else (0, 0, 255)
        for points in self.scaled_segments.values():
            cv2.polylines(image, [points], isClosed=True, color=color, thickness=2)

    def move(self, dx, dy):
        self.dx += dx
        self.dy += dy
        self.scaled_segments = self.recalculate_segments()

    def transform(self, scale_factor):
        self.scale_factor *= scale_factor
        self.scaled_segments = self.recalculate_segments()

    def adjust(self, angle):
        self.angle += angle
        self.scaled_segments = self.recalculate_segments()

    def get_coordinates(self):
        return {segment: points.tolist() for segment, points in self.scaled_segments.items()}

    def handle_mouse_event(self, event, x, y, flags, param, mode):
        if mode == 'select':
            if event == cv2.EVENT_LBUTTONDOWN:
                for points in self.scaled_segments.values():
                    if cv2.pointPolygonTest(points, (x, y), False) >= 0:
                        self.selected = True
                        self.ref_point = (x, y)
                        break
            elif event == cv2.EVENT_LBUTTONUP:
                self.selected = False

        elif mode == 'move':
            if event == cv2.EVENT_LBUTTONDOWN:
                self.ref_point = (x, y)
                self.selected = True
            elif event == cv2.EVENT_MOUSEMOVE and self.selected:
                dx = x - self.ref_point[0]
                dy = y - self.ref_point[1]
                self.move(dx, dy)
                self.ref_point = (x, y)
            elif event == cv2.EVENT_LBUTTONUP:
                self.selected = False

        elif mode == 'transform':
            if event == cv2.EVENT_LBUTTONDOWN:
                self.ref_point = (x, y)
                self.selected = True
            elif event == cv2.EVENT_MOUSEMOVE and self.selected:
                dx = x - self.ref_point[0]
                scale_factor = 1 + dx / 100.0
                self.transform(scale_factor)
                self.ref_point = (x, y)
            elif event == cv2.EVENT_LBUTTONUP:
                self.selected = False

        elif mode == 'adjust':
            if event == cv2.EVENT_LBUTTONDOWN:
                self.ref_point = (x, y)
                self.selected = True
            elif event == cv2.EVENT_MOUSEMOVE and self.selected:
                dx = x - self.ref_point[0]
                angle = dx / 5.0
                self.adjust(angle)
                self.ref_point = (x, y)
            elif event == cv2.EVENT_LBUTTONUP:
                self.selected = False

class SegmentDisplay:
    def __init__(self, image):
        self.original_image = image
        self.temp_image = image.copy()
        self.segments = []
        self.mode = 'select'

    def add_segment(self, segment):
        self.segments.append(segment)

    def update_image(self):
        self.temp_image = self.original_image.copy()
        for segment in self.segments:
            segment.draw(self.temp_image)
        cv2.imshow("image", self.temp_image)

    def run(self):
        cv2.namedWindow("image")
        cv2.setMouseCallback("image", self.handle_mouse_event)
        self.update_image()

        try:
            while True:
                key = cv2.waitKey(1) & 0xFF
                if key == 27:  # ESCキーで終了
                    break
                elif key == ord('s'):  # 's'キーで選択モードに切り替え
                    self.mode = 'select'
                    print("選択モード")
                    self.update_image()
                elif key == ord('m'):  # 'm'キーで移動モードに切り替え
                    self.mode = 'move'
                    print("移動モード")
                    self.update_image()
                elif key == ord('t'):  # 't'キーで変形モードに切り替え
                    self.mode = 'transform'
                    print("変形モード")
                    self.update_image()
                elif key == ord('a'):  # 'a'キーで図形補正モードに切り替え
                    self.mode = 'adjust'
                    print("図形補正モード")
                    self.update_image()
                elif key == ord('c'):  # 'c'キーで現在のセグメントの座標を表示
                    for segment in self.segments:
                        current_coordinates = segment.get_coordinates()
                        print("現在のセグメントの座標:")
                        for seg, coords in current_coordinates.items():
                            print(f"{seg}: {coords}")
                # 他のキー操作を追加可能

        except KeyboardInterrupt:
            print("プログラムが停止されました。")

        cv2.destroyAllWindows()

    def handle_mouse_event(self, event, x, y, flags, param):
        for segment in self.segments:
            segment.handle_mouse_event(event, x, y, flags, param, self.mode)

# 画像の読み込み
image_path = 'C:/Users/Owner/Desktop/my_program/my_pattern_match/image/segment_test.png'
image = cv2.imread(image_path)

# 画像が正しく読み込まれたか確認
if image is None:
    print(f"Error: 画像が見つかりません。パスを確認してください: {image_path}")
else:
    # 7セグメントディスプレイの各セグメントの座標を定義
    segments = {
        'A': np.array([[50, 40], [140, 40], [160, 60], [140, 80], [50, 80], [30, 60]], np.int32),
        'B': np.array([[160, 60], [180, 80], [180, 180], [160, 200], [140, 180], [140, 80]], np.int32),
        'C': np.array([[160, 200], [180, 220], [180, 320], [160, 340], [140, 320], [140, 220]], np.int32),
        'D': np.array([[50, 320], [140, 320], [160, 340], [140, 360], [50, 360], [30, 340]], np.int32),
        'E': np.array([[30, 200], [50, 220], [50, 320], [30, 340], [10, 320], [10, 220]], np.int32),
        'F': np.array([[30, 60], [50, 80], [50, 180], [30, 200], [10, 180], [10, 80]], np.int32),
        'G': np.array([[50, 180], [140, 180], [160, 200], [140, 220], [50, 220], [30, 200]], np.int32),
        'DP': np.array([[180, 360], [200, 360], [200, 380], [180, 380]], np.int32)  # 小数点のドット
    }

    # SegmentDisplayオブジェクトを作成
    display = SegmentDisplay(image)

    # Segmentオブジェクトを作成し、追加
    segment = Segment(segments)
    display.add_segment(segment)

    # 実行
    display.run()

変形モード
移動モード
変形モード
変形モード
変形モード
変形モード
移動モード
変形モード
変形モード
変形モード
変形モード
変形モード
変形モード
移動モード
移動モード


## rev37マウス操作中の描画をましに

In [2]:
import cv2
import numpy as np

class Segment:
    def __init__(self, segments, scale_factor=1.0, dx=0, dy=0, angle=0):
        self.original_segments = segments
        self.scale_factor = scale_factor
        self.dx = dx
        self.dy = dy
        self.angle = angle
        self.scaled_segments = self.recalculate_segments()
        self.selected = False
        self.ref_point = None

    def scale_and_center_segment(self, points, scale_factor, offset_x, offset_y):
        scaled_points = []
        for x, y in points:
            scaled_x = int(x * scale_factor + offset_x)
            scaled_y = int(y * scale_factor + offset_y)
            scaled_points.append([scaled_x, scaled_y])
        return np.array(scaled_points, np.int32)

    def adjust_segment_tilt(self, points, angle):
        center = np.mean(points, axis=0)
        rotation_matrix = cv2.getRotationMatrix2D(tuple(center), angle, 1.0)
        adjusted_points = cv2.transform(np.array([points]), rotation_matrix)[0]
        return adjusted_points

    def recalculate_segments(self):
        g_points = self.original_segments['G']
        g_center = np.mean(g_points, axis=0)
        new_segments = {}
        for segment, points in self.original_segments.items():
            relative_points = points - g_center
            scaled_points = relative_points * self.scale_factor + g_center
            rotated_points = self.adjust_segment_tilt(scaled_points, self.angle)
            new_segments[segment] = self.scale_and_center_segment(rotated_points, 1, self.dx, self.dy)
        return new_segments

    def draw(self, image):
        color = (0, 255, 0) if not self.selected else (0, 0, 255)
        for points in self.scaled_segments.values():
            cv2.polylines(image, [points], isClosed=True, color=color, thickness=2)

    def move(self, dx, dy):
        self.dx += dx
        self.dy += dy
        self.scaled_segments = self.recalculate_segments()

    def transform(self, scale_factor):
        self.scale_factor *= scale_factor
        self.scaled_segments = self.recalculate_segments()

    def adjust(self, angle):
        self.angle += angle
        self.scaled_segments = self.recalculate_segments()

    def get_coordinates(self):
        return {segment: points.tolist() for segment, points in self.scaled_segments.items()}

    def handle_mouse_event(self, event, x, y, flags, param, mode, update_image_callback):
        if mode == 'select':
            if event == cv2.EVENT_LBUTTONDOWN:
                for points in self.scaled_segments.values():
                    if cv2.pointPolygonTest(points, (x, y), False) >= 0:
                        self.selected = True
                        self.ref_point = (x, y)
                        break
            elif event == cv2.EVENT_LBUTTONUP:
                self.selected = False

        elif mode == 'move':
            if event == cv2.EVENT_LBUTTONDOWN:
                self.ref_point = (x, y)
                self.selected = True
            elif event == cv2.EVENT_MOUSEMOVE and self.selected:
                dx = x - self.ref_point[0]
                dy = y - self.ref_point[1]
                self.move(dx, dy)
                self.ref_point = (x, y)
                update_image_callback()
            elif event == cv2.EVENT_LBUTTONUP:
                self.selected = False

        elif mode == 'transform':
            if event == cv2.EVENT_LBUTTONDOWN:
                self.ref_point = (x, y)
                self.selected = True
            elif event == cv2.EVENT_MOUSEMOVE and self.selected:
                dx = x - self.ref_point[0]
                scale_factor = 1 + dx / 100.0
                self.transform(scale_factor)
                self.ref_point = (x, y)
                update_image_callback()
            elif event == cv2.EVENT_LBUTTONUP:
                self.selected = False

        elif mode == 'adjust':
            if event == cv2.EVENT_LBUTTONDOWN:
                self.ref_point = (x, y)
                self.selected = True
            elif event == cv2.EVENT_MOUSEMOVE and self.selected:
                dx = x - self.ref_point[0]
                angle = dx / 5.0
                self.adjust(angle)
                self.ref_point = (x, y)
                update_image_callback()
            elif event == cv2.EVENT_LBUTTONUP:
                self.selected = False

class SegmentDisplay:
    def __init__(self, image):
        self.original_image = image
        self.temp_image = image.copy()
        self.segments = []
        self.mode = 'select'

    def add_segment(self, segment):
        self.segments.append(segment)

    def update_image(self):
        self.temp_image = self.original_image.copy()
        for segment in self.segments:
            segment.draw(self.temp_image)
        cv2.imshow("image", self.temp_image)

    def run(self):
        cv2.namedWindow("image")
        cv2.setMouseCallback("image", self.handle_mouse_event)
        self.update_image()

        try:
            while True:
                key = cv2.waitKey(1) & 0xFF
                if key == 27:  # ESCキーで終了
                    print('図形の追加')
                    self.add_segment(segment)
                # 他のキー操作を追加可能
                    self.update_image()
                elif key == ord('m'):  # 'm'キーで移動モードに切り替え
                    self.mode = 'move'
                    print("移動モード")
                    self.update_image()
                elif key == ord('t'):  # 't'キーで変形モードに切り替え
                    self.mode = 'transform'
                    print("変形モード")
                    self.update_image()
                elif key == ord('a'):  # 'a'キーで図形補正モードに切り替え
                    self.mode = 'adjust'
                    print("図形補正モード")
                    self.update_image()
                elif key == ord('c'):  # 'c'キーで現在のセグメントの座標を表示
                    for segment in self.segments:
                        current_coordinates = segment.get_coordinates()
                        print("現在のセグメントの座標:")
                        for seg, coords in current_coordinates.items():
                            print(f"{seg}: {coords}")
                elif key == ord('+'): # +キーで図形を追加
                    new_segments = {
                        'A': np.array([[50, 40], [140, 40], [160, 60], [140, 80], [50, 80], [30, 60]], np.int32),
                        'B': np.array([[160, 60], [180, 80], [180, 180], [160, 200], [140, 180], [140, 80]], np.int32),
                        'C': np.array([[160, 200], [180, 220], [180, 320], [160, 340], [140, 320], [140, 220]], np.int32),
                        'D': np.array([[50, 320], [140, 320], [160, 340], [140, 360], [50, 360], [30, 340]], np.int32),
                        'E': np.array([[30, 200], [50, 220], [50, 320], [30, 340], [10, 320], [10, 220]], np.int32),
                        'F': np.array([[30, 60], [50, 80], [50, 180], [30, 200], [10, 180], [10, 80]], np.int32),
                        'G': np.array([[50, 180], [140, 180], [160, 200], [140, 220], [50, 220], [30, 200]], np.int32),
                        'DP': np.array([[180, 360], [200, 360], [200, 380], [180, 380]], np.int32)
                    }
                    new_segment = Segment(new_segments)
                    self.add_segment(new_segment)
                    print("新しいセグメントを追加しました")
                    self.update_image()
                # 他のキー操作を追加可能

        except KeyboardInterrupt:
            print("プログラムが停止されました。")

        cv2.destroyAllWindows()

    def handle_mouse_event(self, event, x, y, flags, param):
        for segment in self.segments:
            segment.handle_mouse_event(event, x, y, flags, param, self.mode, self.update_image)


In [3]:

# 画像の読み込み
image_path = 'C:/Users/Owner/Desktop/my_program/my_pattern_match/image/segment_test.png'
image = cv2.imread(image_path)

# 画像が正しく読み込まれたか確認
if image is None:
    print(f"Error: 画像が見つかりません。パスを確認してください: {image_path}")
else:
    # 7セグメントディスプレイの各セグメントの座標を定義
    segments = {
        'A': np.array([[50, 40], [140, 40], [160, 60], [140, 80], [50, 80], [30, 60]], np.int32),
        'B': np.array([[160, 60], [180, 80], [180, 180], [160, 200], [140, 180], [140, 80]], np.int32),
        'C': np.array([[160, 200], [180, 220], [180, 320], [160, 340], [140, 320], [140, 220]], np.int32),
        'D': np.array([[50, 320], [140, 320], [160, 340], [140, 360], [50, 360], [30, 340]], np.int32),
        'E': np.array([[30, 200], [50, 220], [50, 320], [30, 340], [10, 320], [10, 220]], np.int32),
        'F': np.array([[30, 60], [50, 80], [50, 180], [30, 200], [10, 180], [10, 80]], np.int32),
        'G': np.array([[50, 180], [140, 180], [160, 200], [140, 220], [50, 220], [30, 200]], np.int32),
        'DP': np.array([[180, 360], [200, 360], [200, 380], [180, 380]], np.int32)  # 小数点のドット
    }

    # SegmentDisplayオブジェクトを作成
    display = SegmentDisplay(image)

    # Segmentオブジェクトを作成し、追加
    segment = Segment(segments)
    display.add_segment(segment)

    # 実行
    display.run()

移動モード
新しいセグメントを追加しました
変形モード
新しいセグメントを追加しました
移動モード
図形の追加


UnboundLocalError: cannot access local variable 'segment' where it is not associated with a value

: 

## rev38 セグメントを独立

In [ ]:
import cv2
import numpy as np

class Segment:
    def __init__(self, segments, scale_factor=1.0, dx=0, dy=0, angle=0):
        self.original_segments = segments
        self.scale_factor = scale_factor
        self.dx = dx
        self.dy = dy
        self.angle = angle
        self.scaled_segments = self.recalculate_segments()
        self.selected = False
        self.ref_point = None

    def scale_and_center_segment(self, points, scale_factor, offset_x, offset_y):
        scaled_points = []
        for x, y in points:
            scaled_x = int(x * scale_factor + offset_x)
            scaled_y = int(y * scale_factor + offset_y)
            scaled_points.append([scaled_x, scaled_y])
        return np.array(scaled_points, np.int32)

    def adjust_segment_tilt(self, points, angle):
        center = np.mean(points, axis=0)
        rotation_matrix = cv2.getRotationMatrix2D(tuple(center), angle, 1.0)
        adjusted_points = cv2.transform(np.array([points]), rotation_matrix)[0]
        return adjusted_points

    def recalculate_segments(self):
        g_points = self.original_segments['G']
        g_center = np.mean(g_points, axis=0)
        new_segments = {}
        for segment, points in self.original_segments.items():
            relative_points = points - g_center
            scaled_points = relative_points * self.scale_factor + g_center
            rotated_points = self.adjust_segment_tilt(scaled_points, self.angle)
            new_segments[segment] = self.scale_and_center_segment(rotated_points, 1, self.dx, self.dy)
        return new_segments

    def draw(self, image):
        color = (0, 255, 0) if not self.selected else (0, 0, 255)
        for points in self.scaled_segments.values():
            cv2.polylines(image, [points], isClosed=True, color=color, thickness=2)

    def move(self, dx, dy):
        self.dx += dx
        self.dy += dy
        self.scaled_segments = self.recalculate_segments()

    def transform(self, scale_factor):
        self.scale_factor *= scale_factor
        self.scaled_segments = self.recalculate_segments()

    def adjust(self, angle):
        self.angle += angle
        self.scaled_segments = self.recalculate_segments()

    def get_coordinates(self):
        return {segment: points.tolist() for segment, points in self.scaled_segments.items()}

    def handle_mouse_event(self, event, x, y, flags, param, mode, update_image_callback):
        if mode == 'select':
            if event == cv2.EVENT_LBUTTONDOWN:
                for points in self.scaled_segments.values():
                    if cv2.pointPolygonTest(points, (x, y), False) >= 0:
                        self.selected = True
                        self.ref_point = (x, y)
                        break
            elif event == cv2.EVENT_LBUTTONUP:
                self.selected = False

        elif self.selected:  # Only handle events if this segment is selected
            if mode == 'move':
                if event == cv2.EVENT_LBUTTONDOWN:
                    self.ref_point = (x, y)
                elif event == cv2.EVENT_MOUSEMOVE and self.ref_point:
                    dx = x - self.ref_point[0]
                    dy = y - self.ref_point[1]
                    self.move(dx, dy)
                    self.ref_point = (x, y)
                    update_image_callback()
                elif event == cv2.EVENT_LBUTTONUP:
                    self.ref_point = None

            elif mode == 'transform':
                if event == cv2.EVENT_LBUTTONDOWN:
                    self.ref_point = (x, y)
                elif event == cv2.EVENT_MOUSEMOVE and self.ref_point:
                    dx = x - self.ref_point[0]
                    scale_factor = 1 + dx / 100.0
                    self.transform(scale_factor)
                    self.ref_point = (x, y)
                    update_image_callback()
                elif event == cv2.EVENT_LBUTTONUP:
                    self.ref_point = None

            elif mode == 'adjust':
                if event == cv2.EVENT_LBUTTONDOWN:
                    self.ref_point = (x, y)
                elif event == cv2.EVENT_MOUSEMOVE and self.ref_point:
                    dx = x - self.ref_point[0]
                    angle = dx / 5.0
                    self.adjust(angle)
                    self.ref_point = (x, y)
                    update_image_callback()
                elif event == cv2.EVENT_LBUTTONUP:
                    self.ref_point = None

class SegmentDisplay:
    def __init__(self, image):
        self.original_image = image
        self.temp_image = image.copy()
        self.segments = []
        self.mode = 'select'

    def add_segment(self, segment):
        self.segments.append(segment)
        segment.selected = True  # Automatically select the newly added segment

    def update_image(self):
        self.temp_image = self.original_image.copy()
        for segment in self.segments:
            segment.draw(self.temp_image)
        cv2.imshow("image", self.temp_image)

    def run(self):
        cv2.namedWindow("image")
        cv2.setMouseCallback("image", self.handle_mouse_event)
        self.update_image()

        try:
            while True:
                key = cv2.waitKey(1) & 0xFF
                if key == 27:  # ESCキーで終了
                    break
                elif key == ord('s'):  # 's'キーで選択モードに切り替え
                    self.mode = 'select'
                    print("選択モード")
                    self.update_image()
                elif key == ord('m'):  # 'm'キーで移動モードに切り替え
                    self.mode = 'move'
                    print("移動モード")
                    self.update_image()
                elif key == ord('t'):  # 't'キーで変形モードに切り替え
                    self.mode = 'transform'
                    print("変形モード")
                    self.update_image()
                elif key == ord('a'):  # 'a'キーで図形補正モードに切り替え
                    self.mode = 'adjust'
                    print("図形補正モード")
                    self.update_image()
                elif key == ord('c'):  # 'c'キーで現在のセグメントの座標を表示
                    for segment in self.segments:
                        current_coordinates = segment.get_coordinates()
                        print("現在のセグメントの座標:")
                        for seg, coords in current_coordinates.items():
                            print(f"{seg}: {coords}")
                elif key == ord('n'):  # 'n'キーで新しいセグメントを追加
                    new_segments = {
                        'A': np.array([[50, 40], [140, 40], [160, 60], [140, 80], [50, 80], [30, 60]], np.int32),
                        'B': np.array([[160, 60], [180, 80], [180, 180], [160, 200], [140, 180], [140, 80]], np.int32),
                        'C': np.array([[160, 200], [180, 220], [180, 320], [160, 340], [140, 320], [140, 220]], np.int32),
                        'D': np.array([[50, 320], [140, 320], [160, 340], [140, 360], [50, 360], [30, 340]], np.int32),
                        'E': np.array([[30, 200], [50, 220], [50, 320], [30, 340], [10, 320], [10, 220]], np.int32),
                        'F': np.array([[30, 60], [50, 80], [50, 180], [30, 200], [10, 180], [10, 80]], np.int32),
                        'G': np.array([[50, 180], [140, 180], [160, 200], [140, 220], [50, 220], [30, 200]], np.int32),
                        'DP': np.array([[180, 360], [200, 360], [200, 380], [180, 380]], np.int32)
                    }
                    new_segment = Segment(new_segments)
                    self.add_segment(new_segment)
                    print("新しいセグメントを追加しました")
                    self.update_image()

        except KeyboardInterrupt:
            print("プログラムが停止されました。")

        cv2.destroyAllWindows()

    def handle_mouse_event(self, event, x, y, flags, param):
        for segment in self.segments:
            segment.handle_mouse_event(event, x, y, flags, param, self.mode, self.update_image)

# 画像の読み込み
image_path = 'C:/Users/Owner/Desktop/my_program/my_pattern_match/image/segment_test.png'
image = cv2.imread(image_path)

# 画像が正しく読み込まれたか確認
if image is None:
    print(f"Error: 画像が見つかりません。パスを確認してください: {image_path}")
else:
    # SegmentDisplayオブジェクトを作成
    display = SegmentDisplay(image)

    # Segmentオブジェクトを作成し、追加
    initial_segments = {
        'A': np.array([[50, 40], [140, 40], [160, 60], [140, 80], [50, 80], [30, 60]], np.int32),
        'B': np.array([[160, 60], [180, 80], [180, 180], [160, 200], [140, 180], [140, 80]], np.int32),
        'C': np.array([[160, 200], [180, 220], [180, 320], [160, 340], [140, 320], [140, 220]], np.int32),
        'D': np.array([[50, 320], [140, 320], [160, 340], [140, 360], [50, 360], [30, 340]], np.int32),
        'E': np.array([[30, 200], [50, 220], [50, 320], [30, 340], [10, 320], [10, 220]], np.int32),
        'F': np.array([[30, 60], [50, 80], [50, 180], [30, 200], [10, 180], [10, 80]], np.int32),
        'G': np.array([[50, 180], [140, 180], [160, 200], [140, 220], [50, 220], [30, 200]], np.int32),
        'DP': np.array([[180, 360], [200, 360], [200, 380], [180, 380]], np.int32)
    }
    segment = Segment(initial_segments)
    display.add_segment(segment)

    # 実行
    display.run()